<a href="https://colab.research.google.com/github/Takumi173/JPMA2024TF1-2/blob/main/2.%20Smart%20Clinical%20Data%20Validator/2.1.%20Code/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## API Key設定

In [ ]:
# GeminiのAPI Keyを設定する
# 取得方法：https://ai.google.dev/gemini-api/docs?hl=ja

api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

## ファイルとライブラリの準備

In [ ]:
# データとプロトコルのコピー
!git clone https://github.com/cdisc-org/sdtm-adam-pilot-project.git


Cloning into 'sdtm-adam-pilot-project'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 224 (delta 64), reused 220 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (224/224), 24.51 MiB | 2.75 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Updating files: 100% (87/87), done.


In [ ]:
# ライブラリの準備
!pip install -q pypdf google-generativeai

import pypdf
from pypdf import PdfReader

import google.generativeai as genai
from google.colab import userdata

import os
import shutil
import json
import copy
import time
import csv
from typing import List, Dict, Any, Tuple, Optional

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
# 使用するjsonデータとdefine.xmlを新規フォルダにコピーする

source_dir = "sdtm-adam-pilot-project/updated-pilot-submission-package/900172/m5/datasets/cdiscpilot01/tabulations/sdtm"
json_dir   = "json_files"
define_dir = "define_xml"

if not os.path.exists(json_dir):
    os.makedirs(json_dir)

if not os.path.exists(define_dir):
    os.makedirs(define_dir)

for root, _, files in os.walk(source_dir):
  for file in files:
    if file.endswith(".json"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(json_dir, file)
      shutil.copy(source_path, target_path)
    if file.endswith("define.xml"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(define_dir, file)
      shutil.copy(source_path, target_path)

In [ ]:
# jsonファイルをリスト形式で結合した統合ファイル（dataset_list.json）を作成

dataset_list = []
for filename in os.listdir(json_dir):
  if filename.endswith(".json"):
    with open(os.path.join(json_dir, filename), "r") as f:
      try:
        json_data = json.load(f)
        dataset_list.append(json_data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filename}: {e}")

with open("dataset_list.json", "w") as f:
  json.dump(dataset_list, f)


## 症例フィルタリング関数の定義

In [ ]:
def filter_data(data: List[Dict[str, Any]], target_usubjids: List[str]) -> List[Dict[str, Any]]:
    """
    複数のドメインデータを含むリストから、指定されたUSUBJIDのrowsのみを抽出して新しいリストを返す。
    入力データがリストでない場合はエラーメッセージを出力する。
    データ構造は、"columns" 内の "name" が "USUBJID" の列を持つことを前提とする。

    Args:
        data (list): ドメインを結合させたのリスト。リストでない場合はエラーとなる。
        target_usubjids (list): 残したいUSUBJIDのリスト。

    Returns:
        list: フィルタリングされたデータのリスト。
    """
    if not isinstance(data, list):
        print("エラー：入力データはJSONオブジェクトのリストである必要があります。")
        return [] # エラー時は空リストを返す

    if not target_usubjids:
        print("情報：抽出対象のUSUBJIDが指定されていません。元のデータを返します。")
        return copy.deepcopy(data) # 対象がない場合は元のデータのコピーを返す

    filtered_data_list = []
    target_usubjids_set = set(target_usubjids) # 検索効率化のためセットに変換

    for item in data:
        if not isinstance(item, dict):
            print(f"警告: リスト内の要素が辞書型ではありません。スキップします: {item}")
            continue

        usubjid_index = -1
        if 'columns' in item and isinstance(item['columns'], list):
            for i, col in enumerate(item['columns']):
                if isinstance(col, dict) and 'name' in col and col['name'] == 'USUBJID':
                    usubjid_index = i
                    break
        else:
             print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'columns' がないか、リスト形式ではありません。USUBJIDによるフィルタリングはできません。")
             # USUBJIDがない場合はそのまま追加（フィルタリング対象外として）
             filtered_data_list.append(copy.deepcopy(item))
             continue


        if usubjid_index == -1:
            # print(f"警告：データセット '{item.get('fileOID', item.get('itemGroupOID', '不明'))}' に 'name' が 'USUBJID' の列が見つかりません。このデータセットはそのまま追加します。")
            # USUBJID列がないデータセットはフィルタリングせずそのまま追加
            filtered_data_list.append(copy.deepcopy(item))
            continue

        new_data = copy.deepcopy(item) # 元のデータを変更しないようにコピー

        if 'rows' in item and isinstance(item['rows'], list):
            filtered_rows = []
            for row in item['rows']:
                 # rowがリストであり、usubjid_indexが有効な範囲にあるか確認
                if isinstance(row, list) and len(row) > usubjid_index:
                    # USUBJIDが抽出対象に含まれるか確認
                    if row[usubjid_index] in target_usubjids_set:
                        filtered_rows.append(row) # deepcopyはnew_dataで行っているのでrowはそのまま追加
            new_data['rows'] = filtered_rows
            # records数も更新
            if 'records' in new_data:
                 new_data['records'] = len(filtered_rows)
        # 'rows'がない、またはリストでない場合、'rows'と'records'はそのまま（コピー済み）
        elif 'rows' in item:
             print(f"警告：データセット '{new_data.get('fileOID', new_data.get('itemGroupOID', '不明'))}' の 'rows' がリスト形式ではありません。行のフィルタリングはスキップされました。")
        # 'rows'自体がない場合も特に何もしない（コピー済み）


        filtered_data_list.append(new_data)


    return filtered_data_list

# --- 実行テスト用コメント ---
# with open('dataset_list.json', 'r') as f:
#   data = json.load(f)
#
# target_ids = ['01-701-1211']
# filtered_list = filter_data(data, target_ids)
#
# output_filename = 'filtered_list.json'
# with open(output_filename, 'w') as f:
#   json.dump(filtered_list, f, indent=2)
#
# print(f"処理完了：'{output_filename}' に USUBJID が {target_ids} のデータを出力しました。")
# print(f"フィルタリング後のデータセット数: {len(filtered_list)}")
# if filtered_list:
#     print(f"最初のデータセットのレコード数: {filtered_list[0].get('records')}")

## データ書き換え関数の定義

In [ ]:
def data_update(data: List[Dict[str, Any]], target_domain: str, target_usubjid: str, target_seq: int | None, target_variable: str, new_value: Any) -> List[Dict[str, Any]]:
    """
    指定されたUSUBJIDを持つレコードの指定された変数を書き換えます。
    {target_domain}SEQが存在する場合はそれもキーとして使用します。
    元のデータは変更せず、新しいデータ構造を返します。

    Args:
        data (list): データ全体のリスト。指定されたtarget_domainのデータセットを含むことを想定します。
        target_domain (str): 対象のドメイン名（例: "CM"）。
        target_usubjid (str): 書き換えたいレコードのUSUBJID。
        target_seq (int or None): 書き換えたいレコードの{target_domain}SEQの値。
                                   SEQカラムが存在し、絞り込みに使用する場合に値を指定。
                                   SEQでの絞り込みが不要な場合やSEQカラムがない場合はNoneを指定。
        target_variable (str): 書き換えたい変数の名前（例: "CMTRT"）。
        new_value (any): 新しい変数の値。

    Returns:
        list: 指定された変数が更新された新しいデータ全体のリスト。
              該当するレコードが見つからなかった場合、元のデータのコピーを返します。
    """
    updated_data = [] # 更新後のデータリストを格納
    seqname = target_domain + 'SEQ'
    found_target_domain = False # 対象ドメインが見つかったかどうかのフラグ
    update_occurred = False # 実際に更新が行われたかどうかのフラグ

    for dataset in data:
        # 元のデータセットをディープコピーして変更に備える
        updated_dataset = dataset

        # データセットが辞書型であり、itemGroupOIDがターゲットドメインと一致するか確認
        if isinstance(updated_dataset, dict) and updated_dataset.get("itemGroupOID") == target_domain:
            found_target_domain = True # 対象ドメインが見つかった
            columns = updated_dataset.get("columns", [])
            rows = updated_dataset.get("rows", [])

            # columns と rows が期待する型か確認
            if not isinstance(columns, list) or not isinstance(rows, list):
                print(f"警告: '{target_domain}' データセットの 'columns' または 'rows' の形式が不正です。スキップします。")
                updated_data.append(updated_dataset) # 不正な形式でもリストには追加
                continue

            usubjid_index = -1
            seq_index = -1
            variable_index = -1
            has_seq_column = False

            # 各列のインデックスを検索
            for i, col in enumerate(columns):
                if isinstance(col, dict): # 列定義が辞書型か確認
                    col_name = col.get("name")
                    if col_name == "USUBJID":
                        usubjid_index = i
                    elif col_name == seqname:
                        seq_index = i
                        has_seq_column = True
                    elif col_name == target_variable:
                        variable_index = i

            # 必要な列が見つかったか確認
            if usubjid_index == -1:
                print(f"警告: '{target_domain}' データセットに 'USUBJID' 列が見つかりませんでした。このデータセットの更新はスキップされます。")
                updated_data.append(updated_dataset)
                continue
            if variable_index == -1:
                print(f"警告: '{target_domain}' データセットに書き換え対象の変数 '{target_variable}' 列が見つかりませんでした。このデータセットの更新はスキップされます。")
                updated_data.append(updated_dataset)
                continue

            # 行データを更新
            updated_rows = []
            row_updated_in_this_dataset = False # このデータセット内で更新があったか
            for row in rows:
                 # rowがリストであることを確認
                if not isinstance(row, list):
                    print(f"警告: '{target_domain}' データセットにリスト形式でない行が含まれています。スキップします: {row}")
                    updated_rows.append(row) # 不正な行もそのまま追加
                    continue

                # 更新対象の行か判定
                usubjid_match = (len(row) > usubjid_index and row[usubjid_index] == target_usubjid)

                # SEQカラムが存在し、target_seqが指定されている場合のみSEQで絞り込む
                seq_match = True # デフォルトはTrue
                if has_seq_column and target_seq is not None:
                    if seq_index == -1:
                         # これは上の列検索で検出されるはずだが念のため
                        print(f"警告: '{target_domain}' データセットに '{seqname}' 列が見つかりましたがインデックスが無効です。USUBJIDのみで照合します。")
                    elif len(row) > seq_index:
                        # rowの長さが足りていて、SEQ値が一致するか
                        # target_seqが数値であることを期待しているが、型変換は行わない（呼び出し元で適切に指定）
                        seq_match = (row[seq_index] == target_seq)
                    else:
                        # rowの長さが足りない場合は不一致
                        seq_match = False


                # USUBJIDが一致し、かつ(必要な場合は)SEQも一致した場合に更新
                if usubjid_match and seq_match:
                    if len(row) > variable_index:
                        # 行をコピーして変更（updated_datasetはdeepcopyしたが、個々のrowは共有されている可能性があるため）
                        updated_row = list(row)
                        original_value = updated_row[variable_index]
                        updated_row[variable_index] = new_value
                        updated_rows.append(updated_row) # 更新した行を追加

                        # 更新メッセージの表示
                        key_info = f"USUBJID '{target_usubjid}'"
                        if has_seq_column and target_seq is not None and seq_index != -1:
                            key_info += f", '{seqname}' '{target_seq}'"
                        print(f"{key_info} の '{target_variable}' を '{original_value}' から '{new_value}' に更新しました。")

                        row_updated_in_this_dataset = True
                        update_occurred = True # 全体で更新があったフラグを立てる
                    else:
                         # 行の長さが足りない場合は更新できない
                        print(f"警告: USUBJID '{target_usubjid}' (SEQ '{target_seq}') の行で、変数 '{target_variable}' (index={variable_index}) が範囲外です。更新できませんでした。行データ: {row}")
                        updated_rows.append(list(row)) # 元の行（のコピー）を追加
                else:
                    # 更新対象でない行はそのまま（のコピー）を追加
                    updated_rows.append(list(row))

            # 更新後の行リストでデータセットを更新
            updated_dataset["rows"] = updated_rows
            # レコード数も更新 (任意)
            if "records" in updated_dataset:
                updated_dataset["records"] = len(updated_rows)

        # 更新されたデータセット（または元のコピー）を結果リストに追加
        updated_data.append(updated_dataset)

    # ループ終了後、対象ドメインが一つも見つからなかった場合にメッセージ表示
    if not found_target_domain:
        print(f"警告: 対象ドメイン '{target_domain}' を含むデータセットが見つかりませんでした。")


    return updated_data

# --- 書き換えテスト用コメント ---
# # 事前に filtered_data_list が定義されている想定
# updated_data = data_update(filtered_data_list, "DM", "01-701-1211", None, "AGE", 49) # DMにはSEQがないのでNone
# updated_data = data_update(updated_data, "CM", "01-701-1211", 3, "CMTRT", "New Drug 123456789")
# updated_data = data_update(updated_data, "CM", "01-701-1211", 1, "CMDOSE", 123) # SEQ=1は存在しないはず
# updated_data = data_update(updated_data, "VS", "01-701-1211", 5, "VSORRES", "130/80")
#
# # 結果確認（任意）
# with open('updated_data_test.json', 'w') as f:
#     json.dump(updated_data, f, indent=2)
# print("書き換えテストデータを出力しました: updated_data_test.json")

## データ比較関数の定義

In [ ]:
def compare_data(old_data: List[Dict[str, Any]], new_data: List[Dict[str, Any]]) -> None:
    """
    2つのデータリスト（SDTM JSON形式を想定）の更新差分を人間が読みやすい形式で出力します。
    USUBJIDと、ドメイン名+'SEQ'（存在する場合）をキーとして行を比較します。

    Args:
        old_data: 旧データリスト。
        new_data: 新データリスト。
    """

    def get_item_key(item: Dict[str, Any]) -> Optional[str]:
        """データセットの識別子（itemGroupOIDまたはfileOID）を取得"""
        return item.get("itemGroupOID", item.get("fileOID"))

    def create_row_dict(item_group: Dict[str, Any], row: List[Any]) -> Optional[Dict[str, Any]]:
        """行データを列名と値の辞書に変換する"""
        if not isinstance(item_group.get("columns"), list) or not isinstance(row, list):
            return None # 不正な形式の場合はNoneを返す

        row_dict = {}
        columns = item_group["columns"]
        # 列数と行の要素数が一致しない場合への対応
        min_len = min(len(columns), len(row))
        for i in range(min_len):
            col_def = columns[i]
            if isinstance(col_def, dict) and "name" in col_def:
                row_dict[col_def["name"]] = row[i]
        if len(columns) != len(row):
             print(f"警告: {get_item_key(item_group)} で列数({len(columns)})と行要素数({len(row)})が不一致です。行: {row[:10]}...") # 長い行データは省略
        return row_dict

    def get_row_primary_key(item_group_oid: Optional[str], row_dict: Dict[str, Any]) -> Optional[Tuple]:
        """行を一意に識別するキー（タプル）を生成する"""
        if not item_group_oid or 'USUBJID' not in row_dict or row_dict['USUBJID'] is None:
            return None # キーが特定できない場合はNone

        key_values = [('USUBJID', row_dict['USUBJID'])]
        seq_key_name = f"{item_group_oid}SEQ"

        # SEQキーが存在するかどうかをcolumnsから確認（row_dictだけだと欠損の場合に区別できない）
        columns = next((item.get("columns", []) for item in old_data + new_data if get_item_key(item) == item_group_oid), [])
        has_seq_col = any(isinstance(col, dict) and col.get("name") == seq_key_name for col in columns)

        if has_seq_col:
            # SEQ列がある場合、row_dictに値があればキーに含める。なければNoneをキーの一部とする
            seq_value = row_dict.get(seq_key_name)
            key_values.append((seq_key_name, seq_value))

        # タプルに変換して返す（辞書のキーとして使えるように）
        # キーの順序を固定するためにソートする
        return tuple(sorted(key_values))

    def format_primary_key(primary_key: Tuple) -> str:
        """キーのタプルを人間が読みやすい文字列に整形する"""
        return ", ".join(f"{k}='{v}'" for k, v in primary_key)

    # --- データ準備 ---
    old_data_map: Dict[str, Dict[Tuple, Dict[str, Any]]] = {}
    new_data_map: Dict[str, Dict[Tuple, Dict[str, Any]]] = {}
    all_group_oids = set()

    # 旧データをマップに格納
    for item in old_data:
        group_oid = get_item_key(item)
        if not group_oid or not isinstance(item.get("rows"), list):
            continue
        all_group_oids.add(group_oid)
        if group_oid not in old_data_map:
            old_data_map[group_oid] = {}
        for row in item["rows"]:
            row_dict = create_row_dict(item, row)
            if row_dict:
                primary_key = get_row_primary_key(group_oid, row_dict)
                if primary_key:
                    old_data_map[group_oid][primary_key] = row_dict

    # 新データをマップに格納
    for item in new_data:
        group_oid = get_item_key(item)
        if not group_oid or not isinstance(item.get("rows"), list):
            continue
        all_group_oids.add(group_oid)
        if group_oid not in new_data_map:
            new_data_map[group_oid] = {}
        for row in item["rows"]:
            row_dict = create_row_dict(item, row)
            if row_dict:
                primary_key = get_row_primary_key(group_oid, row_dict)
                if primary_key:
                    new_data_map[group_oid][primary_key] = row_dict

    # --- 差分比較と出力 ---
    print("--- データ比較結果 ---")
    change_found = False
    for group_oid in sorted(list(all_group_oids)):
        old_rows = old_data_map.get(group_oid, {})
        new_rows = new_data_map.get(group_oid, {})

        old_keys = set(old_rows.keys())
        new_keys = set(new_rows.keys())

        added_keys = new_keys - old_keys
        removed_keys = old_keys - new_keys
        common_keys = old_keys & new_keys
        updated_keys = {key for key in common_keys if old_rows[key] != new_rows[key]}

        if added_keys or removed_keys or updated_keys:
            change_found = True
            print(f"\n=== ItemGroup: {group_oid} ===")

            # 追加されたデータ
            if added_keys:
                print("\n  (+) 追加された行:")
                for key in sorted(list(added_keys)):
                    print(f"    - キー: {format_primary_key(key)}")

            # 削除されたデータ
            if removed_keys:
                print("\n  (-) 削除された行:")
                for key in sorted(list(removed_keys)):
                    print(f"    - キー: {format_primary_key(key)}")

            # 更新されたデータ
            if updated_keys:
                print("\n  (*) 更新された行:")
                for key in sorted(list(updated_keys)):
                    print(f"    - キー: {format_primary_key(key)}")
                    old_row_dict = old_rows[key]
                    new_row_dict = new_rows[key]
                    all_item_keys = sorted(list(set(old_row_dict.keys()) | set(new_row_dict.keys())))
                    for item_key in all_item_keys:
                        old_value = old_row_dict.get(item_key)
                        new_value = new_row_dict.get(item_key)
                        if old_value != new_value:
                            print(f"        {item_key}: {old_value!r} -> {new_value!r}")

    if not change_found:
        print("旧データと新データの間に差分は見つかりませんでした。")
    print("\n--- 比較終了 ---")


# --- 比較テスト用コメント ---
# # 事前に dataset_list (旧データ) と dataset_list_updated (新データ) が定義されている想定
# compare_data(dataset_list, dataset_list_updated)

## 更新差分の特定関数を定義

In [ ]:
def extract_row(data: List[Dict[str, Any]], target_domain: str, target_usubjid: str, target_seq: int | None) -> Optional[List[Any]]:
    """
    指定されたUSUBJIDとSEQを持つレコード（行データ）を抽出します。
    最初に見つかった行のコピーを返します。

    - {target_domain}SEQ カラムが存在する場合:
        - target_seq が None でない場合: USUBJID と SEQ の両方が一致する行を検索します。
        - target_seq が None の場合: USUBJID のみが一致する行を検索します（SEQカラムの値は無視）。
    - {target_domain}SEQ カラムが存在しない場合:
        - target_seq の値に関わらず、USUBJID のみが一致する行を検索します。

    Args:
        data (list): データ全体のリスト。指定されたtarget_domainのデータセットを含むことを想定します。
        target_domain (str): 対象のドメイン名（例: "CM"）。
        target_usubjid (str): 抽出したいレコードのUSUBJID。
        target_seq (int or None): 抽出したいレコードの{target_domain}SEQの値。
                                   SEQカラムが存在する場合に、SEQでの絞り込みを行う場合に指定します。
                                   Noneを指定すると、SEQカラムの有無に関わらずUSUBJIDのみで検索します。

    Returns:
        list or None: 抽出された行データのコピー (リスト形式)。該当するレコードが見つからなかった場合はNone。
                      最初に見つかった行のみを返します。
                      ※元のデータに影響を与えないよう、見つかった行はコピーして返します。
    """
    seqname = target_domain + 'SEQ'

    for dataset in data:
        # 対象のドメインか確認
        if isinstance(dataset, dict) and dataset.get("itemGroupOID") == target_domain:
            columns = dataset.get("columns", [])
            rows = dataset.get("rows", [])

            # columns と rows が期待する型か確認
            if not isinstance(columns, list) or not isinstance(rows, list):
                print(f"警告: '{target_domain}' データセットの 'columns' または 'rows' の形式が不正です。スキップします。")
                continue # 次の dataset へ

            usubjid_index = -1
            seq_index = -1
            has_seq_column = False

            # 列名からインデックスを検索
            for i, col in enumerate(columns):
                 if isinstance(col, dict): # 列定義が辞書型か確認
                    col_name = col.get("name")
                    if col_name == "USUBJID":
                        usubjid_index = i
                    elif col_name == seqname:
                        seq_index = i
                        has_seq_column = True # SEQカラムが見つかった

            # USUBJID列が存在するかチェック
            if usubjid_index == -1:
                print(f"警告: '{target_domain}' データセットに 'USUBJID' 列が見つかりませんでした。このデータセットはスキップします。")
                continue # 次の dataset へ

            # --- 行データの検索 ---
            for row in rows:
                 # rowがリストであることを確認
                if not isinstance(row, list):
                    print(f"警告: '{target_domain}' データセットにリスト形式でない行が含まれています。スキップします: {row}")
                    continue

                # 行が短すぎてUSUBJIDが取得できない場合はスキップ
                if len(row) <= usubjid_index:
                    continue

                # 1. USUBJIDが一致するか確認
                usubjid_match = (row[usubjid_index] == target_usubjid)

                # USUBJIDが一致しない場合は、この行は対象外
                if not usubjid_match:
                    continue

                # 2. SEQでの絞り込みが必要か判断し、実行
                seq_match = True # デフォルトはTrue (SEQ絞り込み不要、または条件に合致)

                # SEQカラムが存在し、有効なインデックスがあり、かつ target_seq が指定されている場合のみ
                # SEQによる絞り込みを行う必要がある
                needs_seq_match = has_seq_column and seq_index != -1 and target_seq is not None

                if needs_seq_match:
                    # SEQによる絞り込みが必要な場合、実際に値が一致するか確認
                    # 行が短すぎてSEQ値が取得できない場合も不一致とする
                    if len(row) > seq_index:
                        # target_seq と row[seq_index] の型が異なる可能性も考慮すべきだが、
                        # ここでは単純比較を行う（呼び出し元で型を合わせる想定）
                        seq_match = (row[seq_index] == target_seq)
                    else:
                        seq_match = False # 行にSEQ値がないため不一致

                # 3. 最終的な判定
                # USUBJIDが一致し、かつ (必要な場合は) SEQも一致した場合にのみ行を返す
                if seq_match: # usubjid_match は既に確認済み
                    # 元のデータに影響を与えないよう、行データをコピーして返す
                    return copy.deepcopy(row) # list()ではなくdeepcopyを使う

            # --- 対象ドメイン内の全行を検索したが、一致する行が見つからなかった場合 ---
            # このデータセット内には該当行がなかった（他のデータセットに同じドメインがある可能性は低いと想定し、ここで処理を終えることが多いが、厳密には全データセットを見るべき）
            print(f"情報: '{target_domain}' データセット内で指定条件に一致する行が見つかりませんでした。")
            # 該当ドメインは見つかったが、一致する行がなかったので None を返す
            return None # この関数は最初に見つかったものを返す仕様なので、ここでNoneを返して良い

    # ループがすべて終了した場合（data内の全datasetを確認した場合）、対象のドメイン自体が見つからなかった
    print(f"警告: 対象ドメイン '{target_domain}' を含むデータセットが見つかりませんでした。")
    return None

# --- 実行テスト用コメント ---
# # 事前に dataset_list_updated と Target_data が定義されている想定
# unique_target_keys = sorted(list({ (item[0], item[1], item[2]) for item in Target_data }))
#
# print("\n--- 更新データの抽出確認 ---")
# for domain, usubjid, seq in unique_target_keys[:5]: # 最初の5件だけ表示
#     extracted = extract_row(dataset_list_updated, domain, usubjid, seq)
#     if extracted:
#         # print(f"抽出成功: Domain={domain}, USUBJID={usubjid}, SEQ={seq}")
#         # リスト要素を文字列に変換して結合
#         print(",".join(map(str, extracted)))
#     else:
#         print(f"抽出失敗: Domain={domain}, USUBJID={usubjid}, SEQ={seq}")
# print("--- 確認終了 ---")

## PDFをテキスト化する関数を定義

In [ ]:
def extract_text_with_pypdf(pdf_path, start_page=None, end_page=None):
    """
    pypdfを使用してPDFファイルからテキストを抽出します。
    特定のページ範囲を指定できます。

    Args:
        pdf_path (str): PDFファイルのパス。
        start_page (int, optional): 抽出を開始するページ番号 (1始まり)。デフォルトはNone (最初のページから)。
        end_page (int, optional): 抽出を終了するページ番号 (1始まり、このページも含む)。デフォルトはNone (最後のページまで)。

    Returns:
        str: 抽出されたテキスト。エラー時はNone。
    """
    text = ""
    try:
        reader = PdfReader(pdf_path)
        num_pages = len(reader.pages)

        # --- ページ範囲の決定 ---
        # ページ番号は1始まり、インデックスは0始まりなので調整
        start_index = 0
        if start_page is not None:
            # 1始まりのページ番号を0始まりのインデックスに変換
            start_index = max(0, start_page - 1)

        end_index = num_pages
        if end_page is not None:
            # 1始まりのページ番号を0始まりのインデックスに変換し、
            # スライスでそのページまで含めるように +1 する必要はない
            # (例: 250ページまでならインデックス249まで。スライスは[start:end]でendは含まない)
            # なので、end_pageが指定されたら、それがスライスの終了位置
            end_index = min(num_pages, end_page)

        # 開始インデックスが終了インデックス以上、または範囲が無効な場合はエラー
        if start_index >= end_index or start_index >= num_pages:
             print(f"エラー: 無効なページ範囲 (開始={start_page}, 終了={end_page}, 総ページ数={num_pages})")
             return None # または空文字列 "" を返すか、例外を発生させる

        # --- 指定範囲のページからテキストを抽出 ---
        print(f"ページ {start_index + 1} から {end_index} まで抽出中...") # ユーザー向けの表示は1始まり
        for i in range(start_index, end_index):
            page = reader.pages[i]
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n" # ページ間に改行を入れる

        return text

    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません - {pdf_path}")
        return None
    except Exception as e:
        print(f"エラー: PDF処理中に予期せぬエラーが発生しました - {e}")
        return None

## DatasetJSONをCSV変換する関数を定義

In [ ]:
def datasetjson2csv(dataset_list, output_directory):
  # --- データセットごとに処理 ---
  for dataset in dataset_list:
      # itemGroupOIDを取得 (ファイル名に使用)
      item_group_oid = dataset.get('itemGroupOID')
      if not item_group_oid:
          print("警告: 'itemGroupOID' が見つからないデータセットをスキップします。")
          continue

      # CSVファイル名を生成
      csv_filename = os.path.join(output_directory, f"{item_group_oid}.csv")

      # 列名（ヘッダー）を取得
      columns_info = dataset.get('columns', [])
      if not columns_info:
          print(f"警告: {item_group_oid} の 'columns' が見つからないか空です。ヘッダーなしで処理を試みます。")
          header = []
      else:
          # 'name' キーが存在するか確認しながらヘッダーを抽出
          header = [col.get('name', f'unknown_col_{i}') for i, col in enumerate(columns_info)]

      # データ行を取得
      rows_data = dataset.get('rows', [])
      if not rows_data and header: # ヘッダーはあるがデータがない場合
          print(f"情報: {item_group_oid} にはデータ行 ('rows') がありません。ヘッダーのみのファイルを作成します。")
      elif not rows_data and not header: # ヘッダーもデータもない場合
          print(f"警告: {item_group_oid} には列情報もデータ行もありません。空のファイルを作成します。")


      # CSVファイルに書き込み
      try:
          # encoding='utf-8-sig' を指定してExcelでの文字化けを防ぐ
          with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
              writer = csv.writer(csvfile)

              # ヘッダーを書き込み (ヘッダーリストが空でない場合)
              if header:
                  writer.writerow(header)

              # データ行を書き込み (データ行リストが空でない場合)
              if rows_data:
                  writer.writerows(rows_data)

          print(f"CSVファイルを作成しました: {csv_filename}")

      except IOError as e:
          print(f"ファイル書き込みエラー ({csv_filename}): {e}")
      except csv.Error as e:
          print(f"CSV書き込みエラー ({csv_filename}): {e}")
      except Exception as e:
          print(f"予期せぬエラーが発生しました ({csv_filename}): {e}")

  print(f"\n処理が完了しました。CSVファイルは '{output_directory}' フォルダに出力されました。")

# プロトコルの読み取り

In [ ]:
# --- PDFファイルパス定義 ---
PDF_FILE_PATH = "/content/sdtm-adam-pilot-project/updated-pilot-submission-package/900172/m5/53-clin-stud-rep/535-rep-effic-safety-stud/5351-stud-rep-contr/cdiscpilot01/cdiscpilot01.pdf"

# --- 抽出するページ範囲を指定 ---
START_PAGE_NUM = 154
END_PAGE_NUM = 250

# --- PDFからテキストを抽出 ---
print("--- PDFテキスト抽出開始 ---")

# ページ範囲を指定して関数を呼び出す
protocol_text = extract_text_with_pypdf(
    PDF_FILE_PATH,
    start_page=START_PAGE_NUM,
    end_page=END_PAGE_NUM
)

print(f"'{os.path.basename(PDF_FILE_PATH)}' の {START_PAGE_NUM}-{END_PAGE_NUM} ページからテキストを抽出しました (約 {len(protocol_text)} 文字)。")
print("--- PDFテキスト抽出終了 ---")
print(f"\n--- 抽出されたテキスト ---")
print(protocol_text[:500] + "...\n\n******（中略）******\n\n..." + protocol_text[-500:])


--- PDFテキスト抽出開始 ---
ページ 154 から 250 まで抽出中...
'cdiscpilot01.pdf' の 154-250 ページからテキストを抽出しました (約 171466 文字)。
--- PDFテキスト抽出終了 ---

--- 抽出されたテキスト ---
Xanomeline (LY246708) H2Q-MC-LZZT(c) Copyright  2006 Eli Lilly and Company   
Clinical Study Protocol Document Page 1 
The information contained in this clinical study protocol is  
Copyright  2006 Eli Lilly and Company. 
Xanomeline (LY246708) 
Protocol H2Q-MC-LZZT(c) 
Safety and Efficacy of the Xanomeline 
Transdermal Therapeutic System (TTS) in Patients 
with Mild to Moderate Alzheimer’s Disease 
Xanomeline (LY246708) H2Q-MC-LZZT(c) Copyright  2006 Eli Lilly and Company   
Clinical Study Pr...

******（中略）******

...pplying and wearing this patch, if the patient was 
prescribed a drug for Alzheimer’s disease and was given the choice of this patch or an 
oral pill given twice daily (assume that both formulations are equally effective), would 
you (the caregiver): 
  Insist that the patient receive an oral pill 
  Prefer that the patient rece

# JSONデータの書き換え

## 書き換えるデータを定義

In [ ]:
# データ書き換えルールの定義
# 形式: [ドメイン名, USUBJID, SEQ (なければNone), 変数名, 新しい値]

Target_data = [
  ["DM", "01-703-1096",None, "AGE", 49],
  ["LB", "01-703-1042",   3, "LBORRES", "135"],
  ["LB", "01-703-1042",   4, "LBORRES", "145"],
  ["LB", "01-703-1086",  37, "LBORRES", "1"],
  ["LB", "01-703-1086",  72, "LBORRES", "1.2"],
  ["LB", "01-703-1086", 102, "LBORRES", "1.1"],
  ["LB", "01-703-1086", 132, "LBORRES", "1"],
  ["LB", "01-703-1086", 162, "LBORRES", "1.3"],
  ["LB", "01-703-1086", 197, "LBORRES", "0.9"],
  ["LB", "01-703-1086", 232, "LBORRES", "0.8"],
  ["LB", "01-703-1042",   3, "LBSTRESC", "135"],
  ["LB", "01-703-1042",   4, "LBSTRESC", "145"],
  ["LB", "01-703-1086",  37, "LBSTRESC", "1"],
  ["LB", "01-703-1086",  72, "LBSTRESC", "1.2"],
  ["LB", "01-703-1086", 102, "LBSTRESC", "1.1"],
  ["LB", "01-703-1086", 132, "LBSTRESC", "1"],
  ["LB", "01-703-1086", 162, "LBSTRESC", "1.3"],
  ["LB", "01-703-1086", 197, "LBSTRESC", "0.9"],
  ["LB", "01-703-1086", 232, "LBSTRESC", "0.8"],
  ["LB", "01-703-1042",   3, "LBSTRESN", 135],
  ["LB", "01-703-1042",   4, "LBSTRESN", 145],
  ["LB", "01-703-1086",  37, "LBSTRESN", 1],
  ["LB", "01-703-1086",  72, "LBSTRESN", 1.2],
  ["LB", "01-703-1086", 102, "LBSTRESN", 1.1],
  ["LB", "01-703-1086", 132, "LBSTRESN", 1],
  ["LB", "01-703-1086", 162, "LBSTRESN", 1.3],
  ["LB", "01-703-1086", 197, "LBSTRESN", 0.9],
  ["LB", "01-703-1086", 232, "LBSTRESN", 0.8],
  ["LB", "01-703-1042",   3, "LBNRIND", "HIGH"],
  ["LB", "01-703-1042",   4, "LBNRIND", "HIGH"],
  ["LB", "01-703-1086",  37, "LBNRIND", "LOW"],
  ["LB", "01-703-1086",  72, "LBNRIND", "LOW"],
  ["LB", "01-703-1086", 102, "LBNRIND", "LOW"],
  ["LB", "01-703-1086", 132, "LBNRIND", "LOW"],
  ["LB", "01-703-1086", 162, "LBNRIND", "LOW"],
  ["LB", "01-703-1086", 197, "LBNRIND", "LOW"],
  ["LB", "01-703-1086", 232, "LBNRIND", "LOW"],
  ["MH", "01-701-1097",   1, "MHTERM", "LOSS OF CONSCIOUSNESS (PASSED OUT)"],
  ["MH", "01-701-1097",   1, "MHLLT", "LOSS OF CONSCIOUSNESS"],
  ["MH", "01-701-1097",   1, "MHDECOD", "LOSS OF CONSCIOUSNESS"],
  ["MH", "01-701-1097",   1, "MHBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["MH", "01-701-1097",   1, "MHSTDTC", "2013-01-01"],
  ["MH", "01-701-1111",   1, "MHTERM", "HEARING LOSS"],
  ["MH", "01-701-1180",   1, "MHTERM", "DEPRESSION (ANXIETY)"],
  ["MH", "01-701-1180",   1, "MHLLT", "ANXIETY DEPRESSION"],
  ["MH", "01-701-1180",   1, "MHDECOD", "DEPRESSION"],
  ["MH", "01-701-1180",   1, "MHBODSYS", "PSYCHIATRIC DISORDERS"],
  ["MH", "01-702-1082",   1, "MHTERM", "PREMENSTRUAL PAIN"],
  ["MH", "01-702-1082",   1, "MHLLT", "PREMENSTRUAL PAIN"],
  ["MH", "01-702-1082",   1, "MHDECOD", "PREMENSTRUAL PAIN"],
  ["MH", "01-702-1082",   1, "MHBODSYS", "REPRODUCTIVE SYSTEM AND BREAST DISORDERS"],
  ["MH", "01-703-1076",   1, "MHTERM", "ATRIOVENTRICULAR BLOCK (SCHEDULED CARDIAC PACEMAKER INSERTION)"],
  ["MH", "01-703-1076",   1, "MHLLT", "ATRIOVENTRICULAR BLOCK"],
  ["MH", "01-703-1076",   1, "MHDECOD", "ATRIOVENTRICULAR BLOCK"],
  ["MH", "01-703-1076",   1, "MHBODSYS", "CARDIAC DISORDERS"],
  ["MH", "01-703-1279",   1, "MHTERM", "SCHIZOPHRENIFORM DISORDERS"],
  ["MH", "01-703-1279",   1, "MHLLT", "SCHIZOPHRENIFORM DISORDER"],
  ["MH", "01-703-1279",   1, "MHDECOD", "SCHIZOPHRENIFORM DISORDER"],
  ["MH", "01-703-1279",   1, "MHBODSYS", "PSYCHIATRIC DISORDERS"],
  ["MH", "01-703-1299",   1, "MHTERM", "CYCLOTHYMIC DISORDER"],
  ["MH", "01-703-1299",   1, "MHLLT", "CYCLOTHYMIC DISORDER"],
  ["MH", "01-703-1299",   1, "MHDECOD", "CYCLOTHYMIC DISORDER"],
  ["MH", "01-703-1299",   1, "MHBODSYS", "PSYCHIATRIC DISORDERS"],
  ["VS", "01-701-1047",  17, "VSORRES", "121"],
  ["VS", "01-701-1047",  18, "VSORRES", "124"],
  ["VS", "01-701-1047",  66, "VSORRES", "185"],
  ["VS", "01-701-1047",  67, "VSORRES", "183"],
  ["VS", "01-701-1383",  37, "VSORRES", "98"],
  ["VS", "01-701-1383", 122, "VSORRES", "160"],
  ["VS", "01-701-1387",   1, "VSORRES", "146"],
  ["VS", "01-701-1387",  32, "VSORRES", "72"],
  ["VS", "01-701-1047",  17, "VSSTRESC", "121"],
  ["VS", "01-701-1047",  18, "VSSTRESC", "124"],
  ["VS", "01-701-1047",  66, "VSSTRESC", "185"],
  ["VS", "01-701-1047",  67, "VSSTRESC", "183"],
  ["VS", "01-701-1383",  37, "VSSTRESC", "98"],
  ["VS", "01-701-1383", 122, "VSSTRESC", "160"],
  ["VS", "01-701-1387",   1, "VSSTRESC", "146"],
  ["VS", "01-701-1387",  32, "VSSTRESC", "72"],
  ["VS", "01-701-1047",  17, "VSSTRESN", 121],
  ["VS", "01-701-1047",  18, "VSSTRESN", 124],
  ["VS", "01-701-1047",  66, "VSSTRESN", 185],
  ["VS", "01-701-1047",  67, "VSSTRESN", 183],
  ["VS", "01-701-1383",  37, "VSSTRESN", 98],
  ["VS", "01-701-1383", 122, "VSSTRESN", 160],
  ["VS", "01-701-1387",   1, "VSSTRESN", 146],
  ["VS", "01-701-1387",  32, "VSSTRESN", 72],
  ["EX", "01-701-1148",   2, "EXDOSE", 82],
  ["EX", "01-701-1148",   3, "EXDOSE", 216],
  ["EX", "01-703-1258",   2, "EXDOSE", 27],
  ["CM", "01-701-1146",  29, "CMTRT", "PAROXETINE"],
  ["QS", "01-701-1023",1010, "QSORRES", "PRESENT"],
  ["QS", "01-701-1023",1012, "QSORRES", "PRESENT"],
  ["QS", "01-701-1111",5004, "QSORRES", "0"],
  ["QS", "01-701-1111",5019, "QSORRES", "0"],
  ["QS", "01-701-1111",5012, "QSORRES", "0"],
  ["QS", "01-701-1111",5027, "QSORRES", "0"],
  ["QS", "01-701-1118",6002, "QSORRES", "MARKED IMPROVEMENT"],
  ["QS", "01-701-1118",6003, "QSORRES", "MARKED WORSENING"],
  ["QS", "01-701-1181",4018, "QSORRES", "Y"],
  ["QS", "01-701-1181",4058, "QSORRES", "Y"],
  ["QS", "01-701-1181",4019, "QSORRES", "Y"],
  ["QS", "01-701-1181",4059, "QSORRES", "Y"],
  ["QS", "01-701-1181",4020, "QSORRES", "Y"],
  ["QS", "01-701-1023",1010, "QSSTRESC", "2"],
  ["QS", "01-701-1023",1012, "QSSTRESC", "2"],
  ["QS", "01-701-1111",5004, "QSSTRESC", "0"],
  ["QS", "01-701-1111",5019, "QSSTRESC", "0"],
  ["QS", "01-701-1111",5012, "QSSTRESC", "0"],
  ["QS", "01-701-1111",5027, "QSSTRESC", "0"],
  ["QS", "01-701-1118",6002, "QSSTRESC", "1"],
  ["QS", "01-701-1118",6003, "QSSTRESC", "7"],
  ["QS", "01-701-1181",4018, "QSSTRESC", "1"],
  ["QS", "01-701-1181",4058, "QSSTRESC", "1"],
  ["QS", "01-701-1181",4019, "QSSTRESC", "1"],
  ["QS", "01-701-1181",4059, "QSSTRESC", "1"],
  ["QS", "01-701-1181",4020, "QSSTRESC", "1"],
  ["QS", "01-701-1023",1010, "QSSTRESN", 2],
  ["QS", "01-701-1023",1012, "QSSTRESN", 2],
  ["QS", "01-701-1111",5004, "QSSTRESN", 0],
  ["QS", "01-701-1111",5019, "QSSTRESN", 0],
  ["QS", "01-701-1111",5012, "QSSTRESN", 0],
  ["QS", "01-701-1111",5027, "QSSTRESN", 0],
  ["QS", "01-701-1118",6002, "QSSTRESN", 1],
  ["QS", "01-701-1118",6003, "QSSTRESN", 7],
  ["QS", "01-701-1181",4018, "QSSTRESN", 1],
  ["QS", "01-701-1181",4058, "QSSTRESN", 1],
  ["QS", "01-701-1181",4019, "QSSTRESN", 1],
  ["QS", "01-701-1181",4059, "QSSTRESN", 1],
  ["QS", "01-701-1181",4020, "QSSTRESN", 1],
  ["QS", "01-701-1118",6001, "QSDTC", "2014-07-08"],
  ["QS", "01-701-1118",6001, "QSDY", 119],
  ["AE", "01-701-1015",   3, "AESER", "Y"],
  ["AE", "01-701-1015",   3, "AESHOSP", "Y"],
  ["AE", "01-701-1015",   3, "AESTDTC", "2014-01-11"],
  ["AE", "01-701-1015",   3, "AEENDTC", "2014-01-09"],
  ["AE", "01-701-1015",   3, "AESTDY", 10],
  ["AE", "01-701-1015",   3, "AEENDY", 8],
  ["AE", "01-701-1028",   1, "AETERM", "PARKINSON'S DISEASE"],
  ["AE", "01-701-1028",   1, "AELLT", "PARKINSON'S DISEASE"],
  ["AE", "01-701-1028",   1, "AEDECOD", "PARKINSON'S DISEASE"],
  ["AE", "01-701-1028",   1, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-701-1028",   1, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-701-1028",   1, "AESTDTC", "2013-07-01"],
  ["AE", "01-701-1028",   1, "AESTDY", -17],
  ["AE", "01-701-1034",   2, "AETERM", "MALIGNANT HYPERTENSION"],
  ["AE", "01-701-1034",   2, "AELLT", "MALIGNANT HYPERTENSION"],
  ["AE", "01-701-1034",   2, "AEDECOD", "MALIGNANT HYPERTENSION"],
  ["AE", "01-701-1034",   2, "AEBODSYS", "VASCULAR DISORDERS"],
  ["AE", "01-701-1034",   2, "AESOC", "VASCULAR DISORDERS"],
  ["AE", "01-701-1047",   4, "AETERM", "HYPERTENSION"],
  ["AE", "01-701-1047",   4, "AELLT", "HYPERTENSION"],
  ["AE", "01-701-1047",   4, "AEDECOD", "HYPERTENSION"],
  ["AE", "01-701-1047",   4, "AEBODSYS", "VASCULAR DISORDERS"],
  ["AE", "01-701-1047",   4, "AESOC", "VASCULAR DISORDERS"],
  ["AE", "01-701-1363",   1, "AESTDTC", "2013-06-15"],
  ["AE", "01-701-1363",   1, "AEENDTC", "2013-06-14"],
  ["AE", "01-701-1363",   1, "AESTDY", 17],
  ["AE", "01-701-1363",   1, "AEENDY", 16],
  ["AE", "01-701-1047",   3, "AEENDTC", "2013-03-05"],
  ["AE", "01-701-1047",   3, "AEENDY", 22],
  ["AE", "01-701-1383",  12, "AETERM", "BLOOD PRESSURE INCREASED"],
  ["AE", "01-701-1383",  12, "AELLT", "BLOOD PRESSURE INCREASED"],
  ["AE", "01-701-1383",  12, "AEDECOD", "BLOOD PRESSURE INCREASED"],
  ["AE", "01-701-1383",  12, "AEBODSYS", "INVESTIGATIONS"],
  ["AE", "01-701-1383",  12, "AESOC", "INVESTIGATIONS"],
  ["AE", "01-701-1153",   2, "AEACN", "DRUG WITHDRAWN"],
  ["AE", "01-701-1180",   6, "AETERM", "SUDDEN DEATH"],
  ["AE", "01-701-1180",   6, "AELLT", "SUDDEN DEATH"],
  ["AE", "01-701-1180",   6, "AEDECOD", "SUDDEN DEATH"],
  ["AE", "01-701-1180",   6, "AEBODSYS", "GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS"],
  ["AE", "01-701-1180",   6, "AESOC", "GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS"],
  ["AE", "01-703-1258",   2, "AESEV", "SEVERE"],
  ["AE", "01-703-1258",   2, "AESTDTC", "2012-08-01"],
  ["AE", "01-703-1258",   2, "AEENDTC", "2012-10-01"],
  ["AE", "01-703-1258",   2, "AESTDY", 13],
  ["AE", "01-703-1258",   2, "AEENDY", 74],
  ["AE", "01-703-1258",   5, "AESEV", "MODERATE"],
  ["AE", "01-703-1258",   5, "AESER", "Y"],
  ["AE", "01-703-1258",   5, "AEOUT", "RECOVERED/RESOLVED"],
  ["AE", "01-703-1258",   5, "AESLIFE", "Y"],
  ["AE", "01-703-1258",   5, "AESTDTC", "2012-10-02"],
  ["AE", "01-703-1258",   5, "AEENDTC", "2012-12-31"],
  ["AE", "01-703-1258",   5, "AESTDY", 75],
  ["AE", "01-703-1258",   5, "AEENDY", 165],
  ["AE", "01-703-1335",   1, "AETERM", "MULTIPLE SCLEROSIS RELAPSE"],
  ["AE", "01-703-1335",   1, "AELLT", "MULTIPLE SCLEROSIS RELAPSE"],
  ["AE", "01-703-1335",   1, "AEDECOD", "MULTIPLE SCLEROSIS RELAPSE"],
  ["AE", "01-703-1335",   1, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-703-1335",   1, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-703-1335",   1, "AESTDTC", "2014-04-01"],
  ["AE", "01-703-1335",   1, "AEENDTC", "2014-05-01"],
  ["AE", "01-703-1335",   1, "AESTDY", 16],
  ["AE", "01-703-1335",   1, "AEENDY", 46],
  ["AE", "01-703-1403",   2, "AETERM", "MYASTHENIA GRAVIS AGGRAVATED"],
  ["AE", "01-703-1403",   2, "AELLT", "MYASTHENIA GRAVIS AGGRAVATED"],
  ["AE", "01-703-1403",   2, "AEDECOD", "MYASTHENIA GRAVIS"],
  ["AE", "01-703-1403",   2, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-703-1403",   2, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1008",   1, "AETERM", "TREMOR IN HANDS, LEGS"],
  ["AE", "01-704-1008",   1, "AELLT", "TREMOR"],
  ["AE", "01-704-1008",   1, "AEDECOD", "TREMOR"],
  ["AE", "01-704-1008",   1, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1008",   1, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1008",   1, "AEREL", "NONE"],
  ["AE", "01-704-1008",   1, "AESTDTC", "2012-06-01"],
  ["AE", "01-704-1008",   1, "AESTDY", -225],
  ["AE", "01-704-1008",   3, "AETERM", "MUSCLE STIFFNESS"],
  ["AE", "01-704-1008",   3, "AELLT", "MUSCLE STIFFNESS"],
  ["AE", "01-704-1008",   3, "AEDECOD", "MUSCULOSKELETAL STIFFNESS"],
  ["AE", "01-704-1008",   3, "AEBODSYS", "MUSCULOSKELETAL AND CONNECTIVE TISSUE DISORDERS"],
  ["AE", "01-704-1008",   3, "AESOC", "MUSCULOSKELETAL AND CONNECTIVE TISSUE DISORDERS"],
  ["AE", "01-704-1008",   3, "AESTDTC", "2012-06-01"],
  ["AE", "01-704-1008",   3, "AESTDY", -225],
  ["AE", "01-704-1008",   2, "AETERM", "SLOWNESS OF MOVEMENT"],
  ["AE", "01-704-1008",   2, "AELLT", "SLOW MOVEMENT"],
  ["AE", "01-704-1008",   2, "AEDECOD", "BRADYKINESIA"],
  ["AE", "01-704-1008",   2, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1008",   2, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1008",   2, "AESTDTC", "2012-06-01"],
  ["AE", "01-704-1008",   2, "AESTDY", -225],
  ["AE", "01-704-1009",   6, "AETERM", "CHRONIC KIDNEY DISEASE"],
  ["AE", "01-704-1009",   6, "AELLT", "CHRONIC KIDNEY DISEASE"],
  ["AE", "01-704-1009",   6, "AEDECOD", "CHRONIC KIDNEY DISEASE"],
  ["AE", "01-704-1009",   6, "AEBODSYS", "RENAL AND URINARY DISORDERS"],
  ["AE", "01-704-1009",   6, "AESOC", "RENAL AND URINARY DISORDERS"],
  ["AE", "01-704-1009",   6, "AESER", "Y"],
  ["AE", "01-704-1009",   6, "AESLIFE", "Y"],
  ["AE", "01-704-1010",   1, "AETERM", "DIABETES MELLITUS"],
  ["AE", "01-704-1010",   1, "AELLT", "DIABETES MELLITUS"],
  ["AE", "01-704-1010",   1, "AEDECOD", "DIABETES MELLITUS"],
  ["AE", "01-704-1010",   1, "AEBODSYS", "METABOLISM AND NUTRITION DISORDERS"],
  ["AE", "01-704-1010",   1, "AESOC", "METABOLISM AND NUTRITION DISORDERS"],
  ["AE", "01-704-1010",   1, "AESER", "Y"],
  ["AE", "01-704-1010",   1, "AESLIFE", "Y"],
  ["AE", "01-704-1017",   4, "AETERM", "LATE EFFECTS OF CEREBRAL INFARCTION"],
  ["AE", "01-704-1017",   4, "AELLT", "LATE EFFECTS OF CEREBRAL INFARCTION"],
  ["AE", "01-704-1017",   4, "AEDECOD", "CEREBRAL INFARCTION"],
  ["AE", "01-704-1017",   4, "AEBODSYS", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1017",   4, "AESOC", "NERVOUS SYSTEM DISORDERS"],
  ["AE", "01-704-1017",   4, "AESEV", "SEVERE",],
  ["AE", "01-704-1017",   4, "AESTDTC", "2013-10-19"],
  ["AE", "01-704-1017",   4, "AEENDTC", "2013-11-18"],
  ["AE", "01-704-1017",   4, "AESTDY", 14],
  ["AE", "01-704-1017",   4, "AEENDY", 44],
  ["AE", "01-704-1017",   3, "AETERM", "BRAIN DEATH"],
  ["AE", "01-704-1017",   3, "AELLT", "BRAIN DEATH"],
  ["AE", "01-704-1017",   3, "AEDECOD", "BRAIN DEATH"],
  ["AE", "01-704-1017",   3, "AEBODSYS", "GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS"],
  ["AE", "01-704-1017",   3, "AESOC", "GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS"],
  ["AE", "01-704-1017",   3, "AESEV", "SEVERE",],
  ["AE", "01-704-1017",   3, "AESTDTC", "2013-11-18"],
  ["AE", "01-704-1017",   3, "AEENDTC", "2013-11-18"],
  ["AE", "01-704-1017",   3, "AESTDY", 44],
  ["AE", "01-704-1017",   3, "AEENDY", 44],
  ["AE", "01-704-1017",   1, "AEOUT", "RECOVERED/RESOLVED"],
  ["AE", "01-704-1017",   1, "AESTDTC", "2013-10-19"],
  ["AE", "01-704-1017",   1, "AEENDTC", "2013-11-19"],
  ["AE", "01-704-1017",   1, "AESTDY", 14],
  ["AE", "01-704-1017",   1, "AEENDY", 45],
  ["AE", "01-704-1017",   1, "AEACN", "DRUG WITHDRAWN"]
]

## データの書き換え実行と確認

In [ ]:
# --- データ書き換え実行 ---
print("--- データ書き換え処理開始 ---")

dataset_list_original = copy.deepcopy(dataset_list)
dataset_list_updated = copy.deepcopy(dataset_list)

update_count = 0
if Target_data: # Target_dataが空でない場合のみ実行
    for rule in Target_data:
        if len(rule) == 5:
            domain, usubjid, seq, variable, value = rule
            # data_update は更新後のリストを返すので、それを次の更新の入力にする
            dataset_list_updated = data_update(dataset_list_updated, domain, usubjid, seq, variable, value)
            update_count += 1
        else:
            print(f"警告: 不正な形式の書き換えルールをスキップしました: {rule}")
    print(f"{update_count} 件の書き換えルールに基づいてデータ更新を試みました。")
else:
    print("情報: 書き換えルール(Target_data)が空のため、データ書き換えは実行されませんでした。")

# --- 更新後データの確認（差分表示）---
if dataset_list_original and dataset_list_updated: # 両方が存在する場合のみ比較
    print("\n--- 更新前後のデータ比較 ---")
    compare_data(dataset_list_original, dataset_list_updated)
else:
    print("更新前後のデータ比較はスキップされました（データ不備のため）。")

print("\n--- データ書き換え処理終了 ---")

# --- 更新後データをファイルに保存 ---
UPDATED_JSON_FILE = "dataset_list_updated.json"

with open(UPDATED_JSON_FILE, "w", encoding='utf-8') as f:
    json.dump(dataset_list_updated, f, ensure_ascii=False, indent=2)
print(f"\n更新後のデータを '{UPDATED_JSON_FILE}' に保存しました。")

# 更新データをCSVファイルに出力
csv_dir = "updated_files_csv"
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

print("\n更新後のデータをCSVに変換しています...")
datasetjson2csv(dataset_list_updated, csv_dir)

print("\n更新後のデータをZIP圧縮しています...")
!zip -r updated_files_csv.zip /content/updated_files_csv

print("\n処理が完了しました。CSVファイルは 'updated_files_csv.zip' に圧縮されました。")

print("\n--- データ出力処理終了 ---")

--- データ書き換え処理開始 ---
USUBJID '01-703-1096' の 'AGE' を '81' から '49' に更新しました。
USUBJID '01-703-1042', 'LBSEQ' '3' の 'LBORRES' を '14' から '135' に更新しました。
USUBJID '01-703-1042', 'LBSEQ' '4' の 'LBORRES' を '25' から '145' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '37' の 'LBORRES' を '4.56' から '1' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '72' の 'LBORRES' を '8.14' から '1.2' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '102' の 'LBORRES' を '7.45' から '1.1' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '132' の 'LBORRES' を '7.51' から '1' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '162' の 'LBORRES' を '6.13' から '1.3' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '197' の 'LBORRES' を '6.02' から '0.9' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '232' の 'LBORRES' を '5.54' から '0.8' に更新しました。
USUBJID '01-703-1042', 'LBSEQ' '3' の 'LBSTRESC' を '14' から '135' に更新しました。
USUBJID '01-703-1042', 'LBSEQ' '4' の 'LBSTRESC' を '25' から '145' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '37' の 'LBSTRESC' を '4.56' から '1' に更新しました。
USUBJID '01-703-1086', 'LBSEQ' '72' 

# プロンプトの定義

## システムプロンプト

In [ ]:
SysPrompt_single = '''
あなたは、臨床試験データの**統合的レビュー**を支援するAIアシスタントです。主な目的は、**参加者の権利と安全性を保護**し、**試験の科学的妥当性とデータの信頼性を確保**することです。そのために、提供された情報に基づき、**医学的観点からの評価、データ整合性の検証、プロトコル遵守状況の確認**をユーザーの指示に従って行います。

**最重要原則:**
*   **回答の根拠は、提供された情報（JSONデータ、Define.xml、プロトコル）と、あなたが持つ確立された一般的な医学知識の両方とします。** これらに基づき、客観的な事実を記述するとともに、データの医学的な意味合いや潜在的な問題を深く考察してください。
*   **レビューにおいては、参加者の安全性と権利の保護を最優先**してください。潜在的なリスクを示唆する所見には特に注意を払い、積極的に指摘してください。
*   **試験の目的（有効性・安全性の評価）が達成可能か、データは信頼できるか**という観点も常に意識してください。データの不整合やプロトコルからの逸脱が評価結果に与える影響を考慮してください。
*   **医学的な評価や、データ・プロトコルに関する指摘を行う際は、その根拠（データ、プロトコル箇所、医学知識）を明確に示してください。**
*   **提供された情報や確立された一般的な医学知識に基づかない、個人的な意見、想像、推測、ハルシネーションに基づいた情報は生成してはいけません。**

**前提知識:**
*   **SDTM (Study Data Tabulation Model):** CDISCによって策定された臨床試験データの標準モデルです。データはDM, AE, VS, LBなどのドメインに分かれており、レビューにはこれらの**ドメイン情報を横断的・統合的に評価する**必要があります。**統合的な評価**には、これらのドメイン情報を横断的に見る必要があります。
*   **Define.xml:** SDTMデータの構造（変数名、ラベル、コードリスト、データ型など）を記述したメタデータファイルであり、データの意味を正確に理解するために**不可欠な情報源**です。JSONデータの解釈は、**必ずDefine.xmlの定義に基づいて**行ってください。データの意味を正確に理解し、**整合性を検証する上で不可欠**です。
*   **データの不完全性:** 報告されるJSONデータには、データ入力時の間違いや不整合が含まれる可能性があることを理解しています。**不整合や欠損が参加者の安全性や評価の信頼性に影響しないか**を評価する必要があります。
*   **プロトコル:** 臨床試験の実施計画書であり、選択/除外基準、投与計画、評価スケジュール、有効性評価計画（主要/副次評価項目、評価方法、評価時期など）、有害事象報告手順などが規定されています。データのレビューはプロトコル遵守の観点からも行います。**参加者の保護と試験計画の遵守**を確認するための重要な基準です。
*   **医学知識の活用:** あなたが持つ一般的な医学知識（疾患、治療法、薬剤の作用・副作用・相互作用、生理学、検査値の臨床的意義、主要な有効性評価指標の解釈に関する標準的な知識）は、データレビューにおいて重要な役割を果たします。**データの医学的な意味合いを解釈し、潜在的な安全性リスクや有効性に関する疑問点を特定するために不可欠**です。提供されたデータと綿密に照らし合わせ、表面的な情報だけでなく、**参加者の安全性や評価の妥当性に関わる隠れた問題**を指摘するために活用してください。
*   **収集されるデータ:** 主要/副次評価項目を評価するために必要なデータのみ収集され、**適格性を確認するためだけのデータ（スクリーニング時の選択/除外基準の判定のみに使用するデータは）は収集されません。**

**タスク実行における注意:**
*   指定された**出力フォーマット**に厳密に従ってください。
*   **データの参照方法:**
    *   **医療機関への問い合わせ文面（日本語・英語）内**でデータに言及する場合： **Define.xmlで定義された変数に対応する「ラベル名」を使用**し、「ラベル名が「値」ですが...」のような自然な文章で記述してください。
*   提供された情報や一般的な医学知識をもってしてもタスクを実行できない場合（例：必要な情報が欠けている、矛盾が解決できない、専門性が高すぎる判断が必要な場合）、その旨を明確に指摘してください。

**エラーハンドリング:**
*   JSONデータ、Define.xml、またはプロトコルの形式が不正である、あるいは内容が著しく不足しておりレビューが困難な場合は、具体的な問題点を指摘し、処理を中断してください。例：「エラー：Define.xmlファイルが提供されていません。」、「エラー：JSONデータの[ドメイン名]に必要な変数[変数名]が含まれていません。」
'''

## ユーザープロンプト

In [ ]:
UserInput_single = '''**役割:**

あなたは、臨床試験データの**統合的レビュー担当者**です。**メディカルモニター(Medical Monitor)、クリニカルデータマネージャー(DM)、臨床開発モニター(CRA)の視点を併せ持ち**、以下の指示に従って、提供される情報（プロトコル、JSONデータ、Define.xml）をレビューしてください。主な目的は、**「参加者の権利と安全性の保護」**と**「試験の科学的妥当性とデータの信頼性確保（＝プロトコルで目的とした評価が正しく行える状態か）」**を確認することです。そのために、**医学的な観点からの評価、データの整合性チェック、プロトコル遵守状況の確認**を統合的に行い、疑義事項の特定とクエリ/内部確認事項の作成（必要な場合）を行ってください。

**指示:**

**1. 症例サマリーの作成:**

*   参照情報: JSONデータ、Define.xml
*   タスク:
    *   JSONデータとDefine.xmlを参照し、患者の主要なイベントを時系列でまとめたサマリーを作成してください。
    *   患者背景: 最初にDMドメインから、主要な背景情報（例: 年齢、性別、人種など、**Define.xmlで定義されたラベルを使用**）を記載してください。
    *   イベント推移: 有害事象(AE)、検査値(LB)、バイタルサイン(VS)、主要な有効性評価関連イベント（例：腫瘍評価結果の変動、症状スコアの変化など）について、**異常変動**や**臨床的に注目すべき変化**を中心に記述してください。**判断にあたってはあなたの持つ一般的な医学知識を最大限活用し、些細に見える変化でも医学的に重要となりうる場合は含めてください。**
        *   異常・注目すべき変化の基準(例):
            *   有害事象の発現、重症度・重篤度の変化、転帰 (**特に医学的に予期せぬ事象やパターン、参加者の安全性に関わるもの**)
            *   検査値・バイタルサインの基準値からの逸脱 (CTCAE Gradeの変化や明らかな異常値、**特定の疾患や薬剤の影響を示唆するパターン、安全性評価上重要なもの**)
            *   ベースラインからの著しい変動 (**臨床的な意味合いを考慮して判断**)
            *   正常範囲上限/下限付近での臨床的に意味のある変動 (**他のデータとの関連性を考慮**)
            *   有効性評価結果の重要な変化（例：RECIST評価の変更、スコアの閾値超え、**期待される効果からの逸脱や矛盾した結果、評価の信頼性に関わるもの**)
        *   省略:**あなたの医学的判断に基づき、臨床的に明らかに意義がないと考えられる変動**は記載しないでください。
    *   日時: 各イベントの日時は、関連する日付変数（例：AESTDY, LBDY, VSDY, RSDY, QSDY など、Define.xml参照）に基づき特定してください (例: Day 10)。
    *   記述: 簡潔な文章で客観的に記述してください。**ただし、医学的な解釈や懸念を補足する必要がある場合は、括弧書き等で簡潔に追記しても構いません。** (例: `ALT値上昇 (Grade 2, 薬剤性肝障害の可能性を考慮)`)

**2. 統合レビュー:**

*   参照情報: JSONデータ、Define.xml、プロトコル
*   タスク:
    *   以下の**3つの統合レビュー観点**に基づき、提供された情報を注意深く照合し、JSONデータを多角的にレビューしてください。**「参加者の保護」と「評価の信頼性確保」**の観点から、問題点、矛盾、不整合、プロトコルからの逸脱の可能性、潜在的なリスクなどを検出・指摘してください。
    *   特定した各指摘事項に対して、**後述の重要度の定義に基づき重要度（Critical/Major/Minor）を付与してください。** 判断は、**参加者の安全性への潜在的な影響度、および試験の評価項目（有効性・安全性）の信頼性への影響度**を総合的に考慮してください。
    *   各観点内で特定された指摘事項は、重要度が高いもの (Critical > Major > Minor) から順にリストアップし、**観点に応じた連番 (M-1, M-2,... / D-1, D-2,... / P-1, P-2,...)** を付与してください。

*   **統合レビュー観点:**
    *   **【医学的レビュー】 (Medical Monitor視点)**
        *   **安全性評価の妥当性:**
            *   報告された有害事象（AE）の評価（事象名、重篤度、重症度、関連性、処置、転帰）は、他の臨床データ（LB, VS, CM, MH, EX等）や時間経過と照らして医学的に妥当か？ **参加者の安全性に影響を与える可能性のある見落としや評価の誤りはないか？**
            *   検査値（LB）やバイタルサイン（VS）の変動パターンに、**医学的に懸念される点、安全性リスクを示唆する所見はないか？** 基準値内変動でも、特定の傾向や他のデータとの組み合わせがリスクを示唆しないか？
            *   併用薬（CM）と有害事象/既往歴（MH）との関連、潜在的な薬物相互作用について、**一般的な医学知識に基づき、特に注意すべき安全性リスクはないか？**
            *   **データ全体から、未報告の有害事象や安全性シグナルの可能性は示唆されないか？**
        *   **有効性評価の妥当性:**
            *   記録された有効性評価の結果（例: RECIST評価、スコア、バイオマーカー値）は、他の臨床データ（AE, LB, VS, CM, EX等）や時間経過と照らして医学的に見て妥当か？ **説明困難な不整合、期待される薬効や疾患経過からの逸脱、評価の信頼性を損なう可能性のある矛盾はないか？**
            *   複数の有効性評価項目がある場合、それらの結果は医学的に見て一貫しているか？ 大きな乖離がある場合、その理由はデータから推察できるか、あるいはさらなる確認が必要か？
        *   **総合的な医学的判断:** 患者背景、臨床経過全体（安全性・有効性データを含む）を考慮し、**参加者の状態に関する医学的な懸念事項、診断や評価の妥当性に関する疑義はないか？** 個々のデータポイントだけでは見えない、全体像としての問題はないか？

    *   **【データ整合性】 (DM視点)**
        *   **クロスドメイン整合性:** 異なるドメイン間のデータ（日付、ID、関連イベント等）に形式的・論理的な矛盾はないか？ **これらの矛盾が、医学的評価やプロトコル遵守の判断に影響を与えないか？** (例: AE発生日 vs LB/VS測定日、AE回復日 vs LB/VS測定日、AE vs CM開始/終了日、MH vs AE/CM、DM.SEX vs 性別依存のイベント/検査、AE発現日 vs EX投与期間)
        *   **ドメイン内整合性:** 各ドメイン内のデータに形式的・論理的な矛盾はないか？ (例: AE 開始日 <= AE 終了日、投与量と単位の一貫性)
        *   **異常値/外れ値:** 定義された範囲外の値、統計的に極端な値、または現実的にありえない値はないか？ **これらがデータ入力エラーなのか、それとも医学的に重要な情報（【医学的レビュー】で評価）なのかを区別する必要があるか？**
        *   **欠損値:** プロトコル上必須、または医学的・統計的評価に重要な変数に欠損はないか？ **欠損が参加者の安全性評価や試験の有効性評価の信頼性に影響を与えないか？** 欠損が許容されるか、理由が適切か？
        *   **評価基準適用のためのデータ:** 有効性・安全性評価の基準（例: RECIST、CTCAE Grade、スコアリング基準）を適用するために必要な元データは揃っており、計算や評価結果と整合しているか？ **データの不備により評価の信頼性が損なわれていないか？**

    *   **【プロトコル遵守】 (CRA視点)**
        *   **選択/除外基準:** 患者はプロトコルで規定された選択基準を満たし、除外基準に該当していないか？ **基準違反が参加者の安全性や試験結果の解釈に与える影響は？** (DM, MHなどを参照)
        *   **治験薬投与:** 投与レジメン（薬剤、用量、経路、頻度、期間）はプロトコルで規定された通りか？ 投与量の変更・中断・再開は適切に記録されているか？ **逸脱がある場合、参加者の安全性や有効性評価への影響は？** (EXを参照)
        *   **併用禁止/制限薬:** プロトコルで禁止または制限されている薬剤が使用されていないか？ **使用されている場合、参加者の安全性リスクや有効性評価への影響は？** (CMを参照)
        *   **評価スケジュール/手順:** 各評価（安全性、有効性、その他）はプロトコルで規定されたVisit、タイミング（Visit Window含む）、および手順（例: 特定の評価機器、評価方法変数 [--METHOD]）で実施されているか？ **逸脱や欠損がある場合、参加者の安全性監視や評価の信頼性にどのような影響があるか？** (VISIT情報、各ドメインの--DY/VISITNUMなどを参照)
        *   **同意/中止等:** 同意取得日と治験手順開始日の関係、中止基準の遵守、中止理由の記録などは適切か？ **参加者の権利が保護されているか？**

**3. 疑義事項の分類とクエリ/内部確認事項の作成:**

*   参照情報: 統合レビューの結果、JSONデータ、Define.xml、プロトコル
*   タスク:
    *   **統合レビューで特定された問題点や疑義事項についてのみ**、以下の分類を行ってください。
        *   **医療機関へのクエリ:** **参加者の安全性確保、評価の信頼性担保、またはデータの正確性・完全性の確認のために、医療機関への問い合わせが必要**な事項。
        *   **内部確認事項:** 医療機関への問い合わせは不要だが、内部で確認・記録すべき事項（例：軽微なデータ不整合で影響が小さい、解釈に関する内部での議論が必要）。

    *   特定した各指摘事項およびそれに基づくクエリ/内部確認事項に対して、**後述の重要度の定義に基づき重要度（Critical/Major/Minor）を付与してください。** 判断は、**参加者の安全性への潜在的な影響度、および試験の評価項目（有効性・安全性）の信頼性への影響度**を総合的に考慮してください。（この重要度は、関連する統合レビューの指摘事項の重要度と一致させるか、クエリ/確認事項としての影響を再評価して決定してください。）
    *   作成された医療機関へのクエリおよび内部確認事項は、それぞれ重要度が高いもの (Critical > Major > Minor) から順にリストアップし、連番 (Q-1, Q-2,... / I-1, I-2,...) を付与してください。
    *   レビューの結果、クエリや内部確認事項を作成する必要がない場合は、「疑義事項なし」と明確に回答してください。

**重要度の定義:**
*   **Critical (致命的/重大):**
    *   **影響:** **参加者の権利、安全性、健康に**重大なリスク**をもたらす、またはその可能性が極めて高い。** または、**データの信頼性や完全性を著しく損ない**、試験結果（特に**主要評価項目**）の解釈に**重大な影響**を与え、**試験の科学的妥当性を脅かす。** 規制当局への報告義務やGCP遵守に**重大な影響**を与える。**(補足参照: Critical to Quality Factor に関連する問題を含む)**
    *   **具体的な状況例:**
        *   重篤な有害事象 (SAE) の未報告、または評価に関する**医学的に重大な疑義や参加者の安全性を脅かす可能性のある不備**。
        *   主要な選択/除外基準の明確な違反で、**参加者への重大なリスクや主要評価項目の信頼性への重大な影響が懸念される**もの。
        *   治験薬の重大な誤投与で、**医学的に重大な結果を招く可能性が高い**もの。
        *   **主要有効性評価項目**データの欠損、重大な不整合、または信頼性への疑義で、**試験結果の解釈を根本的に覆す可能性がある**もの。
        *   同意取得前の治験関連手順の実施など、**参加者の権利を著しく侵害する**もの。
        *   **データから強く示唆される、未報告の重篤な安全性シグナル。**
    *   **対応:** 通常、即時のアクション（例: 緊急クエリ、プロトコル逸脱報告）が必要。

*   **Major (主要):**
    *   **影響:** **参加者の権利、安全性、健康に**潜在的なリスク**をもたらす可能性がある（Criticalほどではない）。** または、**データの信頼性や完全性に影響**を与え、試験結果（特に**副次評価項目**や重要な安全性評価項目）の解釈に**影響を与える可能性**があり、**評価の信頼性を損なう。** プロトコルからの**重要な逸脱**に該当する。
    *   **具体的な状況例:**
        *   非重篤な有害事象の評価と他の臨床データとの明らかな矛盾で、**医学的な判断や安全性評価に影響を与える可能性がある**もの。
        *   **副次有効性評価項目**や**重要な安全性評価項目**に関連するデータの不整合や欠損、プロトコル規定からの逸脱（評価時期、手順など）で、**評価の信頼性に影響を与える**もの。
        *   併用禁止/制限薬の使用（**医学的な安全性リスクが中程度以下と判断されるが、確認が必要**な場合、または**有効性評価への影響が懸念される**場合）。
        *   重要な検査・評価（安全性・有効性）の未実施や、規定されたVisit Windowからの逸脱で、**医学的評価や評価の信頼性に影響を与える可能性がある**もの。
        *   投与量変更や一時中断/再開に関する記録の不備や矛盾で、**参加者の曝露量評価や安全性評価、有効性評価に影響する**もの。
        *   **データパターンから示唆される、潜在的な安全性懸念や有効性に関する疑問点。**
    *   **対応:** 通常、クエリ発行による確認・修正や、内部での詳細な調査が必要。

*   **Minor (軽微):**
    *   **影響:** **参加者の安全性や試験結果の解釈への**直接的な影響は小さい**と考えられる。** 主にデータの**品質や一貫性**に関わる問題で、**評価の信頼性への影響は限定的**。プロトコルからの**軽微な逸脱**で、試験の主要な目的や医学的評価に大きな影響を与えないもの。
    *   **具体的な状況例:**
        *   明らかな誤字脱字（ただし、事象名や薬剤名、有効性評価の重要なキーワードなど、解釈に影響を与えうる場合はMajor以上と判断することもある）。
        *   重要度の低いデータの欠損や軽微な不整合（例: 終了日が開始日より前だが、臨床的な時間経過から明らかに誤記と判断でき、医学的影響が小さい）。
        *   臨床的に意義の小さい検査値/バイタルサインの記録に関する軽微な矛盾で、**医学的な判断や評価の信頼性に影響しない**もの。
        *   選択/除外基準を判定するための検査結果が報告されておらず、適格性判定が不能な場合。（ただし、他のデータから選択/除外基準違反が強く示唆される場合はMajor以上と判断することもある）
        *   Visit日付のわずかなずれ（プロトコルで許容範囲が定義されていない、または有効性評価の厳密性が低い場合など、**医学的評価や評価の信頼性への影響が無視できる**場合）。
    *   **対応:** 内部確認事項として記録するか、他のクエリと併せて確認する、または修正不要と判断する場合もある。

**補足: Critical to Quality Factor (CtQF) について**
*   CtQF は、試験の品質に不可欠な要素であり、これらに関連する重大な問題は原則として Critical と判断されます。CtQF の例としては以下のようなものが挙げられますが、これらは主に試験全体の品質管理に関わる要素です。個別の症例レビューにおいては、これらの要素が患者の安全性、権利、データの信頼性、特に主要評価項目に与える**具体的な影響度**を考慮して重要度を判断してください。
*   **(CtQF の例 - 主要なもの)**
    *   評価の質（標準化、一貫性）に関する重大な問題
    *   組み入れ/除外基準の厳格な適用に関する重大な違反
    *   併用禁止薬の使用
    *   中止基準違反
    *   主要評価項目に影響を与える重大な欠測やデータ品質の問題
    *   同意プロセスに関する重大な不備


**出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

# [USUBJID]のデータ統合レビュー報告

## 1. 症例サマリー

*   **患者背景:**
主要な背景情報を**Define.xmlのラベル名を用いて**簡潔に記載する。
（例: 68歳、女性、白人（NOT HISPANIC OR LATINO）。治験実施国はUSAであり、実際に割り付けられた治療群はHigh Doseであった。主要な既往歴として、心筋梗塞（1986年発症）、冠動脈バイパス術（2006年実施）が報告されている。）

*   **イベント推移:**

|日付（YYYY年MM月DD日）|Study Day (Visit名)|イベント内容|
|:---|:---|:---|
|YYYY年MM月DD日|Day XX (N/A)|イベント内容 (例: 有害事象「頭痛」(Severe) 発現)|
|YYYY年MM月DD日|Day YY (UNSCHEDULED 1.1)|イベント内容 (例: ALT値上昇 (Grade 1, 基準値上限の1.5倍))|
|YYYY年MM月DD日|Day ZZ (WEEK 2)|イベント内容 (例: RECIST評価: Stable Disease (SD))|
|... (時系列で記載)|...|...|

## 2. 統合レビュー結果

*   **【医学的レビュー】からの指摘事項:**
    *   [指摘事項がない場合は「指摘事項なし」と記載]
    *   (指摘事項がある場合)
        *   **指摘No.:** M-1
            *   **重要度:** [Critical/Major/Minor]
            *   **内容:** [具体的な医学的懸念事項（安全性・有効性含む）、評価の妥当性に関する指摘、潜在的リスクの指摘など。参加者保護や評価信頼性の観点から。]
            *   **根拠:** [判断の根拠、一般的な医学的知見などを記載。]
            *   **関連データ:**
                * [ラベル名(変数名)] = 値（例：[有害事象名(AETERM)] = '頭痛'）
                *   ... (複数の項目が関連する場合は箇条書きで追加する)
        *   ... (複数の指摘事項があれば M-2, M-3...)

*   **【データ整合性】観点からの指摘事項:**
    *   [指摘事項がない場合は「指摘事項なし」と記載]
    *   (指摘事項がある場合)
        *   **指摘No.:** D-1
            *   **重要度:** [Critical/Major/Minor]
            *   **内容:** [具体的なデータの不整合、異常値、欠損値、フォーマットの問題などに関する指摘。その問題が医学的評価や評価の信頼性にどのような影響を持ちうるかも簡潔に触れる。]
            *   **根拠:** [判断の根拠、一般的な医学的知見などを記載。]
            *   **関連データ:**
                * [ラベル名(変数名)] = 値（例：[有害事象開始日(AESTDTC)] = '2023-10-26'）
                *   ... (複数の項目が関連する場合は箇条書きで追加する)
        *   ... (複数の指摘事項があれば D-2, D-3...)

*   **【プロトコル遵守】観点からの指摘事項 (逸脱の可能性):**
    *   [指摘事項がない場合は「指摘事項なし」と記載]
    *   (指摘事項がある場合)
        *   **指摘No.:** P-1
            *   **重要度:** [Critical/Major/Minor]
            *   **逸脱の可能性:** [具体的なプロトコルからの逸脱の可能性。その逸脱が参加者の安全性や評価の信頼性にどのような影響を持ちうるかも簡潔に触れる。]
            *   **プロトコル該当箇所:** [プロトコルの該当するセクション、ページ番号などを記載]
            *   **根拠:** [判断の根拠、一般的な医学的知見などを記載。]
            *   **関連データ:**
                * [ラベル名(変数名)] = 値（例：[投与開始日(CM.CMSTDTC)] = '2023-11-01'）
                *   ... (複数の項目が関連する場合は箇条書きで追加する)
        *   ... (複数の指摘事項があれば P-2, P-3...)

## 3. 疑義事項

*   [クエリも内部確認事項もない場合は「疑義事項なし」と記載]
*   **医療機関へのクエリ:**
    *   [クエリがない場合は「クエリなし」と記載]
    *   (クエリがある場合)
        *   **クエリNo.:** Q-1 (関連指摘No.: [例: M-1, D-2, P-1])
            *   **重要度:** [Critical/Major/Minor]
            *   **発行担当者:** [指摘内容に最も関連性の高い役割を記載: Medical Monitor, CRA, DM]
            *   **医療機関への問い合わせ文面:** [具体的かつ客観的な問い合わせ内容。なぜこの情報が必要なのか、どのような**参加者保護**または**評価信頼性**に関する懸念に基づいているのかを明確に含める。**文中で特定のデータに言及する場合は、Define.xmlで定義された変数に対応する「ラベル名」を使用し、「「ラベル名」が「値」ですが、...」といった形式でラベル名を記述（変数名は不要）してください。例：「有害事象名「頭痛」について、重症度が「高度」と記録されていますが、詳細をお知らせください。」「Study Day 50のアラニンアミノトランスフェラーゼが 「150 IU/L」 と高値ですが、臨床的な意義について評価をお願いします。」**]
            *   **クエリ文面（英語）:** [具体的かつ客観的な医療機関への問い合わせ内容（英語）。なぜこの情報が必要なのか、どのような**参加者保護**または**評価信頼性**に関する懸念に基づいているのかを明確に含めてください。**文中で特定のデータに言及する場合は、可能であればDefine.xmlで定義された変数に対応する英語の「ラベル名」（または平易な英語表現）を使用し、「... the (Label Name / description) was (Value)...」といった形式で簡潔（最大でも300文字）に記述してください。例: "Regarding the AE Term 'Headache', the Severity is recorded as 'Severe'. Please provide further details." / "On Study Day 50, the ALT value was 150 IU/L. Please assess the clinical significance."**]
            *   **判断理由:** [なぜ問い合わせが必要かの簡潔な理由。**参加者の安全性確保、評価の信頼性担保、データの正確性確認**の観点から記載。]
            *   **判断根拠:**
                *   関連するデータ: 例: [検査項目(LB.LBTESTCD)] = 'ALT', [検査日(Study Day)(LB.LBDY)] = 50, [検査結果(数値)(LB.LBSTRESN)] = 150, [毒性グレード(LB.LBTOXGR)] = '2'
                *   関連するプロトコル箇所: 例: Protocol Section Y.Z (安全性モニタリング)
                *   関連する医学的知見: 例: 薬剤性肝障害の可能性
        *   ... (複数のクエリがあれば Q-2, Q-3...)

*   **内部確認事項 (問い合わせ不要):**
    *   [内部確認事項がない場合は「内部確認事項なし」と記載]
    *   (内部確認事項がある場合)
        *   **確認事項No.:** I-1 (関連指摘No.: [例: D-1, P-2])
            *   **重要度:** [Critical/Major/Minor]
            *   **確認担当者:** [指摘内容に最も関連性の高い役割を記載: Medical Monitor, CRA, DM]
            *   **疑義事項/確認内容:** [問い合わせは不要だが、記録・確認すべき内容。**参加者保護や評価信頼性への影響が小さいと判断した理由**も簡潔に記載。]
            *   **判断理由:** [なぜ問い合わせ不要か、なぜ記録が必要かの簡潔な理由。]
            *   **判断根拠:**
                *   関連するデータ: 例: [生年月日(DM.BRTHDTC)] = '1958-05-10', [年齢(DM.AGE)] = 65
                *   関連するプロトコル箇所: 例: Section 4.1 選択基準 (年齢 18-75歳)
        *   ... (複数の内部確認事項があれば I-2, I-3...)
'''



## Define.xmlとプロトコルの埋め込み

In [ ]:
with open('define_xml/define.xml', 'r') as f:
  define_xml = f.read()

UserInput_single_end1 = '\n---\n\n**臨床試験データ（JSON形式、SDTM準拠）:**\n\n```json\n'
UserInput_single_end2 = '\n```\n\n**データ定義ファイル（Define.xml）:**\n\n```xml\n' + define_xml + '\n```\n\n**臨床試験実施計画書（プロトコル）:**\n\n```\n' + protocol_text

# Geminiによるレビューの実行

## パラメータ設定

In [ ]:
# --- Geminiモデル設定 ---
MODEL_NAME = 'gemini-2.5-pro'
#MODEL_NAME = 'gemini-2.5-pro-exp-03-25'  # <- 公開結果を得たときに使用したバージョン
#MODEL_NAME = 'gemini-2.0-flash'
#MODEL_NAME = 'gemini-1.5-flash'

# --- 生成パラメータ設定 ---
GENERATION_TEMPERATURE = 0.2  # 創造性の調整 (低いほど決定的)
GENERATION_TOP_P = 0.8        # トークン選択の確率調整 (高いほど多様)

generation_config = genai.types.GenerationConfig(
    temperature=GENERATION_TEMPERATURE,
    top_p=GENERATION_TOP_P,
    # max_output_tokens=GENERATION_MAX_OUTPUT_TOKENS # 必要に応じて設定
)

genai.configure(api_key=api_key)

print(f"Geminiモデル: {MODEL_NAME}")
print(f"生成設定: Temperature={GENERATION_TEMPERATURE}, Top_P={GENERATION_TOP_P}")
# if 'GENERATION_MAX_OUTPUT_TOKENS' in locals():
#      print(f"最大出力トークン数: {GENERATION_MAX_OUTPUT_TOKENS}")

Geminiモデル: gemini-2.5-pro
生成設定: Temperature=0.2, Top_P=0.8


In [ ]:
# --- 対象の特定 ---
# Target_data から更新があった症例リストを作成
updated_subjects_set = {item[1] for item in Target_data if len(item) > 1}
updated_subjects = sorted(list(updated_subjects_set))

print(f"レビュー対象 ({len(updated_subjects)}名): {updated_subjects}")

# --- 実行対象のAPIと症例の設定 ---
# https://ai.google.dev/gemini-api/docs/rate-limits?hl=ja
# 実行時の制限：'gemini-2.5-pro-exp-03-25' の Limit rate: 250,000 TPM / 1,000,000 TPD
# Input token数が 250k - 350k であり、1症例でTPM Limit、3症例でTPD Limitを迎える可能性があるので実行の分割とループ実行時の待機処理が必要

# 30症例あるためバッチで分割実行
batch = 0

if batch == 0:
    updated_subjects = updated_subjects[0:3]
elif batch == 1:
    updated_subjects = updated_subjects[3:6]
elif batch == 2:
    updated_subjects = updated_subjects[6:9]
elif batch == 3:
    updated_subjects = updated_subjects[9:12]
elif batch == 4:
    updated_subjects = updated_subjects[12:15]
elif batch == 5:
    updated_subjects = updated_subjects[15:18]
elif batch == 6:
    updated_subjects = updated_subjects[18:21]
elif batch == 7:
    updated_subjects = updated_subjects[21:24]
elif batch == 8:
    updated_subjects = updated_subjects[24:27]
elif batch == 9:
    updated_subjects = updated_subjects[27:30]

print(f"実行対象 ({len(updated_subjects)}名): {updated_subjects}")

レビュー対象 (30名): ['01-701-1015', '01-701-1023', '01-701-1028', '01-701-1034', '01-701-1047', '01-701-1097', '01-701-1111', '01-701-1118', '01-701-1146', '01-701-1148', '01-701-1153', '01-701-1180', '01-701-1181', '01-701-1363', '01-701-1383', '01-701-1387', '01-702-1082', '01-703-1042', '01-703-1076', '01-703-1086', '01-703-1096', '01-703-1258', '01-703-1279', '01-703-1299', '01-703-1335', '01-703-1403', '01-704-1008', '01-704-1009', '01-704-1010', '01-704-1017']
実行対象 (3名): ['01-701-1015', '01-701-1023', '01-701-1028']


## Geminiに送る

In [ ]:
# --- リトライ設定 ---
MAX_RETRIES = 10
INITIAL_DELAY = 1 # 秒

# --- TPM制限に対応するためのクールダウン時間 ---
TPM_COOLDOWN = 60 # 秒

print("--- Geminiレビュー実行開始 ---")

results_list = []
processed_count = 0
error_count = 0
failed_subjects = [] # リトライしても失敗した症例IDを格納するリスト

# dataset_list_updated と protocol_text が存在するか確認
if 'dataset_list_updated' not in locals() or not dataset_list_updated:
    print("エラー: 更新後のデータリスト (dataset_list_updated) が存在しません。レビューを実行できません。")
elif 'protocol_text' not in locals():
     print("エラー: プロトコルテキスト (protocol_text) が存在しません。レビューを実行できません。")
elif not updated_subjects:
    print("情報: レビュー対象の被験者がいません。処理をスキップします。")
else:
    # GenerativeModelインスタンスを作成 (システムプロンプトとモデル名を指定)
    try:
        # modelインスタンスの作成はループの外で行う（毎回作成する必要はない）
        model = genai.GenerativeModel(
            model_name=MODEL_NAME,
            system_instruction=SysPrompt_single,
            generation_config=generation_config # 生成設定を渡す
        )
        print(f"モデル '{MODEL_NAME}' の初期化完了。")
    except Exception as e:
        print(f"致命的エラー: モデル '{MODEL_NAME}' の初期化に失敗しました: {e}")
        # モデル初期化失敗時はループを実行しない
        updated_subjects = [] # ループが実行されないように空にする

    # 各被験者についてレビューを実行
    total_subjects = len(updated_subjects)
    for i, subj_id in enumerate(updated_subjects):
        print(f"\n[{i+1}/{total_subjects}] 被験者 '{subj_id}' のレビューを開始...")
        start_time = time.time() # 各被験者の処理開始時間

        # 1. 対象被験者のデータを抽出
        subject_data_list = filter_data(dataset_list_updated, [subj_id])
        if not subject_data_list:
            print(f"  警告: 被験者 '{subj_id}' のデータ抽出に失敗しました。スキップします。")
            # エラーとして記録し、失敗リストに追加
            results_list.append(f"# [{subj_id}] のデータ統合レビュー報告\n\nエラー: 対象データの抽出に失敗しました。")
            error_count += 1
            failed_subjects.append(subj_id + " (データ抽出失敗)")
            continue

        # 2. プロンプトを組み立てる
        try:
            subject_data_json = json.dumps(subject_data_list, ensure_ascii=False)
            prompt_parts = UserInput_single + UserInput_single_end1 + subject_data_json + UserInput_single_end2 + '\n```'

        except Exception as e:
             print(f"  エラー: 被験者 '{subj_id}' のプロンプト組み立て中にエラーが発生しました: {e}")
             # エラーとして記録し、失敗リストに追加
             results_list.append(f"# [{subj_id}] のデータ統合レビュー報告\n\nエラー: プロンプト組み立て失敗: {e}")
             error_count += 1
             failed_subjects.append(subj_id + " (プロンプト組み立て失敗)")
             continue

        # 3. Gemini API呼び出し (リトライロジック付き)
        response = None # response変数を初期化
        last_exception = None # 最後のエラーを保持するため
        for attempt in range(MAX_RETRIES + 1): # 初回実行 + MAX_RETRIES回のリトライ
            try:
                if attempt > 0:
                     print(f"    リトライ {attempt}/{MAX_RETRIES} 回目...")

                api_start_time = time.time()
                response = model.generate_content(prompt_parts) # 部品リストで渡す
                api_end_time = time.time()
                api_elapsed_time = api_end_time - api_start_time

                # ---- 成功時の処理 ----
                # 4. 結果の評価と格納
                if response.text:
                    results_list.append(response.text)
                    total_elapsed_time = time.time() - start_time
                    print(f"  レビュー完了 ({total_elapsed_time:.2f} 秒, API:{api_elapsed_time:.2f} 秒)")
                    processed_count += 1
                    # トークン数の表示（利用可能な場合）
                    if hasattr(response, 'usage_metadata') and response.usage_metadata:
                        print(f"    Tokens: Prompt={response.usage_metadata.prompt_token_count}, Candidates={response.usage_metadata.candidates_token_count}, Total={response.usage_metadata.total_token_count}")
                    else:
                        print("    トークン使用量メタデータは利用できません。")

                    # トークン数の表示（TPMクールダウンタイム）
                    if total_subjects != i +1:
                        print(f"  {TPM_COOLDOWN} 秒待機して次の症例を開始します...")
                        time.sleep(TPM_COOLDOWN)

                    break # ★★★ 成功したらリトライループを抜ける ★★★
                else:
                     # レスポンスはあるがテキストがない場合（ブロックされた等）
                     # これはAPIエラーではないためリトライ対象外とし、警告として記録
                    total_elapsed_time = time.time() - start_time
                    print(f"  警告: 被験者 '{subj_id}' のレビュー結果が空です。({total_elapsed_time:.2f} 秒, API:{api_elapsed_time:.2f} 秒)")
                    # 詳細情報を取得
                    finish_reason = "UNKNOWN"
                    block_reason = "NONE"
                    if response.candidates:
                         candidate = response.candidates[0]
                         finish_reason = candidate.finish_reason.name
                         if hasattr(candidate, 'safety_ratings'):
                             safety_ratings = candidate.safety_ratings
                             block_reason = next((r.category.name for r in safety_ratings if r.probability.name != "NEGLIGIBLE"), "NONE")

                    results_list.append(f"# [{subj_id}] のデータ統合レビュー報告\n\n警告: レビュー結果が空でした。\nFinish Reason: {finish_reason}\nBlock Reason: {block_reason}")
                    error_count += 1
                    failed_subjects.append(subj_id + f" (結果空: {finish_reason}/{block_reason})")
                    break # ★★★ このケースもリトライせずループを抜ける ★★★

            except Exception as e:
                # ---- 例外発生時の処理 ----
                last_exception = e # 最後のエラーを更新
                api_end_time = time.time()
                api_elapsed_time = api_end_time - api_start_time
                print(f"    エラー発生 (試行 {attempt+1}/{MAX_RETRIES+1}, API:{api_elapsed_time:.2f} 秒): {e}")

                if attempt < MAX_RETRIES:
                    # まだリトライ可能な場合
                    RETRY_DELAY = INITIAL_DELAY * 2 ** (attempt)
                    RETRY_DELAY = min(RETRY_DELAY,60)

                    print(f"    {RETRY_DELAY}秒後にリトライします...")
                    time.sleep(RETRY_DELAY)
                    # ループの次の繰り返しへ
                else:
                    # 最大リトライ回数に達した場合
                    print(f"  エラー: 被験者 '{subj_id}' のレビュー中にAPIエラーが{MAX_RETRIES+1}回発生しました。最終エラー: {last_exception}")
                    total_elapsed_time = time.time() - start_time
                    # エラーとして記録し、失敗リストに追加
                    results_list.append(f"# [{subj_id}] のデータ統合レビュー報告\n\nエラー: API呼び出し失敗 ({MAX_RETRIES+1}回試行後): {last_exception}")
                    error_count += 1
                    failed_subjects.append(subj_id + f" (APIエラー: {last_exception})")
                    # リトライループはここで終了（breakは不要、ループ条件で抜ける）

# --- ループ完了後 ---
print(f"\n--- Geminiレビュー実行終了 ---")
print(f"処理完了: {processed_count} 件成功, {error_count} 件エラー/警告")

# 実行できなかった（失敗した）症例リストを表示
if failed_subjects:
    print("\n--- 処理に失敗した症例リスト ---")
    # 重複を除いて表示（データ抽出失敗とAPIエラー両方発生する可能性も考慮）
    unique_failed_subjects = sorted(list(set(failed_subjects)))
    for failed_info in unique_failed_subjects:
        print(f"- {failed_info}")
else:
    print("\nすべての症例の処理が完了しました（失敗なし）。")

--- Geminiレビュー実行開始 ---
モデル 'gemini-2.5-pro' の初期化完了。

[1/3] 被験者 '01-701-1015' のレビューを開始...
  レビュー完了 (149.09 秒, API:145.27 秒)
    Tokens: Prompt=328437, Candidates=4216, Total=340136
  60 秒待機して次の症例を開始します...

[2/3] 被験者 '01-701-1023' のレビューを開始...
  レビュー完了 (136.51 秒, API:131.31 秒)
    Tokens: Prompt=246056, Candidates=4152, Total=255826
  60 秒待機して次の症例を開始します...

[3/3] 被験者 '01-701-1028' のレビューを開始...
  レビュー完了 (135.87 秒, API:128.56 秒)
    Tokens: Prompt=333319, Candidates=4893, Total=343953

--- Geminiレビュー実行終了 ---
処理完了: 3 件成功, 0 件エラー/警告

すべての症例の処理が完了しました（失敗なし）。


In [ ]:
# --- 結果の結合 ---
if 'results_list' in locals() and results_list:
    # 各結果の間に空行を2行入れて結合
    combined_markdown = "\n\n---\n\n".join(results_list)

    # --- ファイルへの保存 ---
    # MODEL_NAME が定義されているか確認
    model_name_suffix = MODEL_NAME.replace("/", "_") if 'MODEL_NAME' in locals() else "unknown_model"
    output_filename = f'output_review_{model_name_suffix}_Batch-{batch}.md'

    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(combined_markdown)
        print(f"\nレビュー結果をMarkdownファイルに保存しました: {output_filename}")
    except Exception as e:
        print(f"エラー: レビュー結果のファイル保存に失敗しました: {e}")

    # --- 結果の表示（任意、長くなる可能性あり） ---
    print(f"\n--- 結合されたレビュー結果 (最初の1000文字) ---")
    print(combined_markdown[:1000] + "...")
    # print(combined_markdown) # 全文表示したい場合

else:
    print("レビュー結果リストが存在しないか空のため、結合・保存・表示は行われませんでした。")


レビュー結果をMarkdownファイルに保存しました: output_review_gemini-2.5-pro_Batch-0.md

--- 結合されたレビュー結果 (最初の1000文字) ---
# 01-701-1015のデータ統合レビュー報告

## 1. 症例サマリー

*   **患者背景:**
63歳、女性、人種は白人、民族はヒスパニックまたはラティーノ。治験実施国はUSAであり、実際に割り付けられた治療群はPlaceboであった。主要な既往歴として、アルツハイマー病（2010年04月30日発症）が報告されている。その他、動悸、子宮部分摘出術(1986年)、頭痛、耳鳴、消化不良、胆石(2012年)、甲状腺部分切除術(1973年)、咽喉頭痛(2013年12月)、扁桃摘出術(1973年)、脚のしびれなどの既往歴がある。

*   **イベント推移:**

|日付（YYYY年MM月DD日）|Study Day (Visit名)|イベント内容|
|:---|:---|:---|
|2013年12月26日|Day -7 (SCREENING 1)|スクリーニング検査実施。Aspartate Aminotransferase (AST)が40 U/Lと軽度高値（基準値上限34）、Alkaline Phosphatase (ALP)が34 U/Lと軽度低値（基準値下限35）を認める。|
|2014年01月02日|Day 1 (BASELINE)|治験薬（プラセボ）投与開始。ベースラインのADAS-Cog(11) Subscoreは13点。|
|2014年01月03日|Day 2 (N/A)|有害事象「APPLICATION SITE ERYTHEMA」（軽度）および「APPLICATION SITE PRURITUS」（軽度）を発現。|
|2014年01月09日|Day 8 (N/A)|有害事象「DIARRHOEA」（軽度）を発現。本有害事象は重篤（入院）と報告されている。|
|2014年01月11日|Day 10 (N/A)|有害事象「DIARRHOEA」が回復。|
|2014年01月16日|Day 15 (WEEK 2)|Alanine Aminotransferase (ALT)が41 U/Lと軽度高値（基準値上限34）を認める。|
|2014年03月05

In [ ]:
# 送付プロンプトの確認
# print(prompt_parts)

包括レビューの実行は以上で完了。

以下は個別の症例に対する問い合わせを行うプログラム。

# 個別の症例に対して問い合わせを行う

以下の実行前に「JSONデータの書き換え」セクションまでを実行すること。

## パラメータ設定

In [ ]:
genai.configure(api_key=api_key)

In [ ]:
# --- Geminiモデル設定 ---
MODEL_NAME = 'gemini-2.5-pro'
#MODEL_NAME = 'gemini-2.5-pro-exp-03-25'
#MODEL_NAME = 'gemini-2.0-flash'
#MODEL_NAME = 'gemini-1.5-flash'

# --- 生成パラメータ設定 ---
GENERATION_TEMPERATURE = 0.2  # 創造性の調整 (低いほど決定的)
GENERATION_TOP_P = 0.8        # トークン選択の確率調整 (高いほど多様)

generation_config = genai.types.GenerationConfig(
    temperature=GENERATION_TEMPERATURE,
    top_p=GENERATION_TOP_P,
    # max_output_tokens=GENERATION_MAX_OUTPUT_TOKENS # 必要に応じて設定
)

## Geminiに送る

In [ ]:
print("\n--- Geminiレビュー実行開始 ---")

# 問い合わせる症例
subj_id = '01-704-1017'

# 問い合わせる内容
prompt = '''以下の点について、それぞれ見解を説明してください。
LATE EFFECTS OF CEREBRAL INFARCTIONのSeriousnessの判定について。
BRAIN DEATHのSeriousnessの判定について。
MYOCARDIAL INFARCTIONの転帰日がBRAIN DEATHより後の日付であることについて。
'''

model = genai.GenerativeModel(
    model_name=MODEL_NAME,
    generation_config=generation_config # 生成設定を渡す
)
print(f"モデル '{MODEL_NAME}' の初期化完了。")

print(f"'{subj_id}' のレビューを開始...")

start_time = time.time() # 各被験者の処理開始時間

# 対象被験者のデータを抽出
subject_data_list = filter_data(dataset_list_updated, [subj_id])
subject_data_json = json.dumps(subject_data_list, ensure_ascii=False)

prompt_parts = prompt + UserInput_single_end1 + subject_data_json + UserInput_single_end2 + '\n```'

api_start_time = time.time()
response = model.generate_content(prompt_parts)
api_end_time = time.time()
api_elapsed_time = api_end_time - api_start_time

# 2. 結果の評価と格納
if response.text:
    total_elapsed_time = time.time() - start_time
    print(f"  レビュー完了 ({total_elapsed_time:.2f} 秒, API:{api_elapsed_time:.2f} 秒)")
    # トークン数の表示（利用可能な場合）
    if hasattr(response, 'usage_metadata') and response.usage_metadata:
        print(f"    Tokens: Prompt={response.usage_metadata.prompt_token_count}, Candidates={response.usage_metadata.candidates_token_count}, Total={response.usage_metadata.total_token_count}")
    else:
        print("    トークン使用量メタデータは利用できません。")

    print(f"\n--- Geminiの回答 ---\n{response.text}")

else:
    total_elapsed_time = time.time() - start_time
    print(f"  警告: 被験者 '{subj_id}' のレビュー結果が空です。({total_elapsed_time:.2f} 秒, API:{api_elapsed_time:.2f} 秒)")
    # 詳細情報を取得
    finish_reason = "UNKNOWN"
    block_reason = "NONE"
    if response.candidates:
          candidate = response.candidates[0]
          finish_reason = candidate.finish_reason.name
          if hasattr(candidate, 'safety_ratings'):
              safety_ratings = candidate.safety_ratings
              block_reason = next((r.category.name for r in safety_ratings if r.probability.name != "NEGLIGIBLE"), "NONE")

    print(f"# [{subj_id}] のデータ統合レビュー報告\n\n警告: レビュー結果が空でした。\nFinish Reason: {finish_reason}\nBlock Reason: {block_reason}")


md_content = f'# 問い合わせ事項 ({subj_id})\n' + prompt + '\n\n# 回答\n' + response.text

with open(f"{subj_id}の見解.md", "w") as f:
  f.write(md_content)


--- Geminiレビュー実行開始 ---
モデル 'gemini-2.5-pro' の初期化完了。
'01-704-1017' のレビューを開始...
  レビュー完了 (75.80 秒, API:71.77 秒)
    Tokens: Prompt=234158, Candidates=1975, Total=239948

--- Geminiの回答 ---
ご提示いただいた臨床試験データ（JSON）、データ定義ファイル（Define.xml）、および臨床試験実施計画書（プロトコル）に基づき、以下の各項目について見解を説明します。

### 1. LATE EFFECTS OF CEREBRAL INFARCTIONのSeriousnessの判定について

**見解：**
データ上、この有害事象の重篤性（Seriousness）の判定は**「プロトコルの定義に基づけば妥当」**と考えられます。しかし、医学的な観点からは慎重な評価が必要です。

**説明：**
提供された有害事象（AE）データセットにおいて、`AETERM`が`LATE EFFECTS OF CEREBRAL INFARCTION`（脳梗塞後遺症）のレコードは以下の通りです。

*   **AETERM (有害事象名):** LATE EFFECTS OF CEREBRAL INFARCTION
*   **AESEV (重症度):** SEVERE (重度)
*   **AESER (重篤性):** N (No, 非重篤)
*   **AEOUT (転帰):** NOT RECOVERED/NOT RESOLVED (未回復/未軽快)
*   **AESTDTC (開始日):** 2013-10-19
*   **AEENDTC (終了日):** 2013-11-18

注目すべき点は、重症度（AESEV）が「重度」であるにもかかわらず、重篤性（AESER）が「非重篤」と判定されている点です。

1.  **プロトコルにおける重篤な有害事象（SAE）の定義:**
    臨床試験実施計画書のセクション `3.9.3.2.2. Serious Adverse Events` によると、SAEは以下のいずれかに該当する場合と定義されています。
    *   死亡に至る (`AESDTH` 

In [ ]:
# 送付プロンプトの確認
# print(prompt_parts)